In [1]:
import open3d as o3d
import numpy as np
from sklearn.cluster import KMeans, DBSCAN
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import pairwise_distances
from sklearn.mixture import GaussianMixture
from sklearn.ensemble import IsolationForest
from sklearn.cluster import Birch
import json
import matplotlib.colors as mcolors
from matplotlib.colors import rgb_to_hsv, hsv_to_rgb

In [2]:
def calculate_darkness(colors):
    """
    Calculate darkness of each point. Assuming colors are in RGB format.
    A simple approach is to average the RGB values, lower values indicating darkness.
    """
    # Convert RGB to grayscale using luminosity method: 0.21 R + 0.72 G + 0.07 B
    grayscale = 0.21 * colors[:, 0] + 0.72 * colors[:, 1] + 0.07 * colors[:, 2]
    return grayscale

def filter_by_darkness(colors, percentile_threshold):
    """
    Filter points by darkness, using a specified percentile threshold.
    """
    darkness = calculate_darkness(colors)
    darkness_threshold = np.percentile(darkness, percentile_threshold)
    # Indices of points darker than the threshold
    return np.where(darkness <= darkness_threshold)[0]

# Adjust brightness of the point cloud
def adjust_brightness(colors, target_brightness):
    current_brightness = np.mean(colors, axis=1)
    brightness_factor = target_brightness / (current_brightness + 1e-3)
    adjusted_colors = colors * brightness_factor[:, np.newaxis]
    return np.clip(adjusted_colors, 0, 1)

def calculate_brightness(colors):
    # Simple average to define brightness
    return np.mean(colors, axis=1)

def adjust_layer_brightness(colors, target_brightness, z_coords, bins):
    adjusted_colors = np.copy(colors)
    # Bin z-coordinates to create layers
    z_bins = np.digitize(z_coords, bins)
    unique_bins = np.unique(z_bins)
    
    for bin in unique_bins:
        # Identify points in the current bin/layer
        in_bin = z_bins == bin
        # Calculate current average brightness of the layer
        current_avg_brightness = np.mean(calculate_brightness(colors[in_bin]))
        # Calculate adjustment factor
        adjustment_factor = target_brightness / current_avg_brightness
        # Apply adjustment
        adjusted_colors[in_bin] *= adjustment_factor
        # Clip to valid range
        adjusted_colors[in_bin] = np.clip(adjusted_colors[in_bin], 0, 255)
    
    return adjusted_colors

# def is_grey(color, threshold=1):
#     """
#     Determine if a color is grey based on the RGB components being within a threshold.
#     """
#     r, g, b = color
#     max_diff = np.max(np.abs(np.array([r, g, b]) - np.mean([r, g, b])))
#     return max_diff <= threshold

def is_grey(colors, threshold=0.02):
    """
    Vectorized check if colors are grey, based on the RGB components being within a threshold.
    """
    # Calculate the mean across the RGB channels
    mean_colors = np.mean(colors, axis=1, keepdims=True)
    # Calculate the maximum absolute difference from the mean for each color
    max_diff = np.max(np.abs(colors - mean_colors), axis=1)
    # Identify grey colors based on the threshold
    grey_mask = max_diff <= threshold
    return grey_mask

def calculate_greyness(colors):
    """
    Calculate the 'greyness' of each color by finding the standard deviation of the RGB components.
    Lower standard deviation means the color is closer to grey.
    """
    # Standard deviation among R, G, B components for each point
    std_deviation = np.std(colors, axis=1)
    return std_deviation

def oversaturate_colors(colors_rgb):
    """
    Oversaturate the colors of the point cloud.
    Colors are assumed to be in [0, 1] range.
    """
    # Convert RGB to HSV
    colors_hsv = rgb_to_hsv(colors_rgb)
    
    # Increase saturation; ensure it does not exceed 1
    colors_hsv[:, 1] = np.clip(colors_hsv[:, 1] * 12, 0, 1)  # Increase saturation by 50%
    
    # Convert back to RGB
    colors_rgb_oversaturated = hsv_to_rgb(colors_hsv)
    
    return colors_rgb_oversaturated

def aggregate_covariances(pcd, radius=0.1, min_neighbors=10):
    """
    Aggregate covariance matrices for each point based on neighbors within a specified radius.
    Only considers points with a minimum number of neighbors to ensure meaningful local geometry.
    """
    kdtree = o3d.geometry.KDTreeFlann(pcd)
    aggregated_covariances = []
    valid_indices = []  # To track indices of points with sufficient neighbors
    for i in range(len(pcd.points)):
        [_, idx, _] = kdtree.search_radius_vector_3d(pcd.points[i], radius)
        if len(idx) >= min_neighbors:  # Check against min_neighbors threshold
            neighbors = np.asarray(pcd.points)[idx]
            cov_matrix = np.cov(neighbors.T)
            aggregated_covariances.append(cov_matrix)
            valid_indices.append(i)
        else:  # Ignore points that do not meet the neighbor threshold
            continue
    return np.array(aggregated_covariances), np.array(valid_indices)

def compute_eigenvalues_and_vectors_from_aggregated_covariances(aggregated_covariances):
    eigenvalues = np.zeros((len(aggregated_covariances), 3))
    eigenvectors = np.zeros((len(aggregated_covariances), 3, 3))
    for i, cov in enumerate(aggregated_covariances):
        e_vals, e_vecs = np.linalg.eigh(cov)
        eigenvalues[i] = e_vals
        eigenvectors[i] = e_vecs
    return eigenvalues, eigenvectors


def detect_bifurcation_points_with_layer_filtering(eigenvalues, valid_indices, layers, distance_percentile, covariance_percentile):
    # Calculate the ratios of the smallest to largest eigenvalues
    ratios = eigenvalues[:, 2] / eigenvalues[:, 0]

    # Calculate the dynamic threshold
    # threshold = np.percentile(ratios, covariance_percentile)

    # Create a mapping from original indices to filtered indices
    index_mapping = {original_index: filtered_index for filtered_index, original_index in enumerate(valid_indices)}

    # Adjust bifurcation_indices to be valid for the ratios array
    adjusted_bifurcation_indices = np.array([index_mapping.get(index, -1) for index in valid_indices])

    # Remove any indices that couldn't be mapped (-1)
    adjusted_bifurcation_indices = adjusted_bifurcation_indices[adjusted_bifurcation_indices != -1]

    # Apply threshold to filter bifurcation points
    is_bifurcation = ratios <= 1.4
    bifurcation_indices = adjusted_bifurcation_indices[is_bifurcation[adjusted_bifurcation_indices]]

    return bifurcation_indices

def detect_bifurcation_points(eigenvalues, valid_indices, percentile_threshold):
    """
    Detect bifurcation points based on the eigenvalues of the aggregated covariances.
    """
    # Calculate the ratios of the smallest to largest eigenvalues
    ratios = eigenvalues[:, 2] / eigenvalues[:, 0]

    # Create a mapping from original indices to filtered indices
    index_mapping = {original_index: filtered_index for filtered_index, original_index in enumerate(valid_indices)}

    # Adjust bifurcation_indices to be valid for the ratios array
    adjusted_bifurcation_indices = np.array([index_mapping.get(index, -1) for index in valid_indices])

    # Remove any indices that couldn't be mapped (-1)
    adjusted_bifurcation_indices = adjusted_bifurcation_indices[adjusted_bifurcation_indices != -1]

    # Apply threshold to filter bifurcation points
    is_bifurcation = ratios <= 1.4
    bifurcation_indices = adjusted_bifurcation_indices[is_bifurcation[adjusted_bifurcation_indices]]

    return bifurcation_indices



def split_into_layers(pcd, num_layers=20):
    """
    Split the point cloud into specified number of z layers.
    Calculate distance from the mean (x, y) position of each layer for each point.
    """
    z_values = np.asarray(pcd.points)[:, 2]
    z_min, z_max = np.min(z_values), np.max(z_values)
    layer_bounds = np.linspace(z_min, z_max, num_layers + 1)
    layers = {}
    for i in range(num_layers):
        layer_indices = np.where((z_values >= layer_bounds[i]) & (z_values < layer_bounds[i+1]))[0]
        if len(layer_indices) > 0:
            layer_points = np.asarray(pcd.points)[layer_indices]
            mean_x, mean_y = np.mean(layer_points[:, 0]), np.mean(layer_points[:, 1])
            # Calculate distance from the layer's mean (x, y) position
            distances = np.sqrt((layer_points[:, 0] - mean_x)**2 + (layer_points[:, 1] - mean_y)**2)
            layers[i] = {
                "indices": layer_indices,
                "distances": distances
            }
    return layers

def filter_points_by_distance(layers, percentile_threshold):
    """
    Filter points in each layer based on their distance from the layer's mean (x, y) position,
    using a percentile threshold.
    """
    valid_points_indices = []
    for layer in layers.values():
        distance_threshold = np.percentile(layer["distances"], percentile_threshold)
        valid_indices = layer["indices"][layer["distances"] <= distance_threshold]
        valid_points_indices.extend(valid_indices)
    return np.array(valid_points_indices)

def calculate_darkness(colors):
    """
    Calculate darkness of each point. Assuming colors are in RGB format.
    A simple approach is to average the RGB values, lower values indicating darkness.
    """
    # Convert RGB to grayscale using luminosity method: 0.21 R + 0.72 G + 0.07 B
    grayscale = 0.21 * colors[:, 0] + 0.72 * colors[:, 1] + 0.07 * colors[:, 2]
    return grayscale

def filter_by_darkness(colors, percentile_threshold):
    """
    Filter points by darkness, using a specified percentile threshold.
    """
    darkness = calculate_darkness(colors)
    darkness_threshold = np.percentile(darkness, percentile_threshold)
    # Indices of points darker than the threshold
    return np.where(darkness <= darkness_threshold)[0]

def find_and_color_neighbors(pcd, bifurcation_indices, num_neighbors=5):
    """
    For each bifurcation point, find and color the nearest neighbors.
    """
    # Convert Open3D PointCloud to NumPy array for easier manipulation
    pcd_points = np.asarray(pcd.points)
    pcd_colors = np.asarray(pcd.colors)
    
    # Ensure colors array exists
    if pcd_colors.shape[0] != pcd_points.shape[0]:
        pcd_colors = np.zeros_like(pcd_points)  # Initialize with zeros if colors are not set

    # Build KDTree for efficient neighbor search
    kdtree = o3d.geometry.KDTreeFlann(pcd)

    for bifurcation_idx in bifurcation_indices:
        # Search for nearest neighbors
        [k, idx, _] = kdtree.search_knn_vector_3d(pcd_points[bifurcation_idx], num_neighbors + 1)  # +1 to include self
        # Color neighbors red, excluding the bifurcation point itself
        for i in range(1, k):  # start from 1 to exclude the bifurcation point itself
            pcd_colors[idx[i]] = [1, 0, 0]  # Red

    # Update colors in the original point cloud
    pcd.colors = o3d.utility.Vector3dVector(pcd_colors)

In [24]:
import numpy as np
import open3d as o3d

def split_into_layers(pcd, num_layers=20):
    """
    Split the point cloud into specified number of z layers.
    Calculate distance from the mean (x, y) position of each layer for each point.
    """
    z_values = np.asarray(pcd.points)[:, 2]
    z_min, z_max = np.min(z_values), np.max(z_values)
    layer_bounds = np.linspace(z_min, z_max, num_layers + 1)
    layers = {}
    for i in range(num_layers):
        layer_indices = np.where((z_values >= layer_bounds[i]) & (z_values < layer_bounds[i+1]))[0]
        if len(layer_indices) > 0:
            layer_points = np.asarray(pcd.points)[layer_indices]
            mean_x, mean_y = np.mean(layer_points[:, 0]), np.mean(layer_points[:, 1])
            # Calculate distance from the layer's mean (x, y) position
            distances = np.sqrt((layer_points[:, 0] - mean_x)**2 + (layer_points[:, 1] - mean_y)**2)
            layers[i] = {
                "indices": layer_indices,
                "distances": distances
            }
    return layers

def filter_points_by_distance(layers, percentile_threshold):
    """
    Filter points in each layer based on their distance from the layer's mean (x, y) position,
    using a percentile threshold.
    """
    valid_points_indices = []
    for layer in layers.values():
        distance_threshold = np.percentile(layer["distances"], percentile_threshold)
        valid_indices = layer["indices"][layer["distances"] <= distance_threshold]
        valid_points_indices.extend(valid_indices)
    return np.array(valid_points_indices)

# Function to detect bifurcation points with layer-based filtering applied only for bifurcation analysis
def detect_bifurcation_points_with_layer_filtering(eigenvalues, valid_indices, layers, percentile_threshold):
    """
    Detect bifurcation points based on eigenvalue ratios, applying layer-based distance filtering
    only for the purpose of bifurcation point analysis.
    """
    epsilon = 1e-8  # To avoid division by zero
    ratios = eigenvalues[:, 2] / (eigenvalues[:, 0] + epsilon)
    valid_ratios = np.isfinite(ratios)
    
    # Filter points based on distance from origin within their Z layer using the specified percentile
    layer_filtered_indices = filter_points_by_distance(layers, percentile_threshold)
    
    # Apply the dynamic threshold based on eigenvalue ratios
    threshold = np.percentile(ratios[valid_ratios], 10)  # Use a dynamic percentile for the threshold
    bifurcation_indices = np.intersect1d(valid_indices, layer_filtered_indices)

    # Correctly filter bifurcation_indices based on the threshold
    # Remove the incorrect double-indexing of ratios
    filtered_ratios = ratios[bifurcation_indices]  # Get the ratios for the filtered indices only once
    bifurcation_indices_filtered = bifurcation_indices[filtered_ratios <= threshold]  # Filter indices based on threshold

    return bifurcation_indices_filtered


# Load and preprocess the point cloud
pcd = o3d.io.read_point_cloud("3d-forest-classic-data/tree_1.pcd")
# pcd = pcd.voxel_down_sample(voxel_size=0.01)

# Noise removal and normal estimation
labels = np.array(pcd.cluster_dbscan(eps=0.1, min_points=5, print_progress=True))
inliers = np.where(labels != -1)
pcd = pcd.select_by_index(inliers[0])
pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))

# Estimate covariances for the entire point cloud
pcd.estimate_covariances(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))

# Split into layers for distance-based filtering
layers = split_into_layers(pcd, num_layers=20)

# Aggregate covariances for each point
aggregated_covariances, valid_indices = aggregate_covariances(pcd, radius=0.1, min_neighbors=10)

# Compute eigenvalues and eigenvectors from the aggregated covariances
eigenvalues, eigenvectors = compute_eigenvalues_and_vectors_from_aggregated_covariances(aggregated_covariances)

# Detect bifurcation points with layer-based filtering
bifurcation_indices = detect_bifurcation_points_with_layer_filtering(eigenvalues, valid_indices, layers, percentile_threshold=50)

# Visualize bifurcation points
colors = np.asarray(pcd.colors)
if len(colors) == 0:
    colors = np.zeros((len(pcd.points), 3))
colors[bifurcation_indices] = [1, 0, 0]  # Red for bifurcation points
pcd.colors = o3d.utility.Vector3dVector(colors)

# Visualize the entire point cloud with highlighted bifurcation points
o3d.visualization.draw_geometries([pcd])

Precompute neighbors.[========================================] 100%


IndexError: index 23393 is out of bounds for axis 0 with size 23393

: 

In [3]:
# Load the point cloud
pcd = o3d.io.read_point_cloud("aligned_tree.pcd")
points = np.asarray(pcd.points)
colors = np.asarray(pcd.colors)

# Adjust brightness if needed
target_brightness = np.mean(calculate_brightness(colors))
adjusted_colors = adjust_brightness(colors, target_brightness)
colors = adjusted_colors

# Calculate greyness for each point
greyness = calculate_greyness(adjusted_colors)

# Determine the 60th percentile greyness threshold
greyness_threshold = np.percentile(greyness, 85)

# Identify grey and non-grey points
grey_mask = is_grey(adjusted_colors, threshold=greyness_threshold)
grey_indices = np.where(grey_mask)[0]
nongrey_indices = np.where(~grey_mask)[0]

# Create and visualize grey points cloud
grey_pcd = o3d.geometry.PointCloud()
grey_pcd.points = o3d.utility.Vector3dVector(points[grey_indices])
grey_pcd.colors = o3d.utility.Vector3dVector(adjusted_colors[grey_indices])

# o3d.visualization.draw_geometries([grey_pcd], window_name="Grey Points")

# Create and visualize non-grey points cloud
nongrey_pcd = o3d.geometry.PointCloud()
nongrey_pcd.points = o3d.utility.Vector3dVector(points[nongrey_indices])
nongrey_pcd.colors = o3d.utility.Vector3dVector(adjusted_colors[nongrey_indices])

kmeans = KMeans(n_clusters=2, random_state=0).fit(adjusted_colors[nongrey_indices])
green_label = np.argmax([np.mean(adjusted_colors[nongrey_indices][kmeans.labels_ == i, 1]) for i in range(2)])

# visualize the green points
green_pcd = o3d.geometry.PointCloud()
green_pcd.points = o3d.utility.Vector3dVector(points[nongrey_indices][kmeans.labels_ == green_label])
green_pcd.colors = o3d.utility.Vector3dVector(adjusted_colors[nongrey_indices][kmeans.labels_ == green_label])
# o3d.visualization.draw_geometries([green_pcd], window_name="Green Points")

# Apply DBSCAN clustering and get labels
labels = np.array(green_pcd.cluster_dbscan(eps=0.01, min_points=7, print_progress=True))

# Create a color map (you can use any color map you like)
color_map = plt.get_cmap("tab10")

# Assign each label a unique color
label_colors = color_map(labels / np.max(labels))

# Create a new point cloud for the labeled points
labeled_pcd = o3d.geometry.PointCloud()
labeled_pcd.points = o3d.utility.Vector3dVector(points[nongrey_indices][kmeans.labels_ == green_label])

# Only take the RGB values of the colors
labeled_pcd.colors = o3d.utility.Vector3dVector(label_colors[:, :3])

# print number of clusters
print(np.max(labels))

# Visualize the labeled points
o3d.visualization.draw_geometries([labeled_pcd], window_name="Labeled Points")

/opt/homebrew/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


Precompute neighbors.[========================================] 100%
619stering[=====================================>  ] 92%
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


In [4]:
pcd = o3d.io.read_point_cloud("aligned_tree.pcd")
points = np.asarray(pcd.points)
colors = np.asarray(pcd.colors)

# Filter points by darkness
filtered_indices = filter_by_darkness(colors, 25)
filtered_points = points[filtered_indices]
filtered_colors = colors[filtered_indices]

# filtered_points = points
# filtered_colors = colors


# remove outliers

# Calculate target brightness
target_brightness = np.mean(np.mean(filtered_colors, axis=0))

# Adjust brightness
adjusted_colors = adjust_brightness(filtered_colors, target_brightness)

colors = adjusted_colors

# Assuming 'colors' are in [0, 1] range. If they're in [0, 255], normalize them first.
colors_normalized = colors / 255.0 if np.max(colors) > 1 else colors

# Oversaturate colors
colors = oversaturate_colors(colors_normalized)

# visualize the adjusted point cloud
adjusted_pcd = o3d.geometry.PointCloud()
adjusted_pcd.points = o3d.utility.Vector3dVector(filtered_points)
adjusted_pcd.colors = o3d.utility.Vector3dVector(colors)
o3d.visualization.draw_geometries([adjusted_pcd], window_name="Adjusted Point Cloud")


# visualize the adjusted point cloud
adjusted_pcd = o3d.geometry.PointCloud()
adjusted_pcd.points = o3d.utility.Vector3dVector(filtered_points)
adjusted_pcd.colors = o3d.utility.Vector3dVector(colors)
# o3d.visualization.draw_geometries([adjusted_pcd], window_name="Adjusted Point Cloud")

# downsample the point cloud
# downsampled_pcd = adjusted_pcd.voxel_down_sample(voxel_size=0.001)
pcd = adjusted_pcd
colors = np.asarray(pcd.colors)
points = np.asarray(pcd.points)

greyness = calculate_greyness(colors)

# Determine the 60th percentile greyness threshold
greyness_threshold = np.percentile(greyness, 30)

# Filter points that are grey based on the greyness threshold
# Points with greyness less than or equal to the threshold are considered grey
grey_mask = greyness <= greyness_threshold
grey_indices = np.where(grey_mask)[0]
nongrey_indices = np.where(~grey_mask)[0]

# Create and visualize grey points cloud
grey_pcd = o3d.geometry.PointCloud()
grey_pcd.points = o3d.utility.Vector3dVector(points[grey_indices])
grey_pcd.colors = o3d.utility.Vector3dVector(colors[grey_indices])  # Convert back if needed

# Visualization for grey points
o3d.visualization.draw_geometries([grey_pcd], window_name="Grey Points")

# Create a new point cloud for non-grey points
nongrey_pcd = o3d.geometry.PointCloud()
nongrey_pcd.points = o3d.utility.Vector3dVector(points[nongrey_indices])
nongrey_pcd.colors = o3d.utility.Vector3dVector(colors[nongrey_indices])

# Visualization
o3d.visualization.draw_geometries([nongrey_pcd], window_name="Non-grey Points")
# labels = np.array(pcd.cluster_dbscan(eps=0.006, min_points=10, print_progress=True))
# max_label = labels.max()
# print(f"point cloud has {max_label + 1} clusters")

# # visualize the clustered point cloud
# clusters = []
# for i in range(max_label + 1):
#     cluster = pcd.select_by_index(np.where(labels == i)[0])
#     if len(cluster.points) > 1000:
#         clusters += [cluster]
# print(f"point cloud has {len(clusters)} clusters")

# cluster_pcd = o3d.geometry.PointCloud()
# cluster_pcd.points = o3d.utility.Vector3dVector(np.concatenate([np.asarray(cluster.points) for cluster in clusters]))
# cluster_pcd.colors = o3d.utility.Vector3dVector(np.concatenate([np.asarray(cluster.colors) for cluster in clusters]))

# classes = 10
# cluster_points = np.asarray(cluster_pcd.points)
# cluster_colors = np.asarray(cluster_pcd.colors)
# kmeans = KMeans(n_clusters=classes, random_state=0).fit(cluster_colors)
# kmeans_labels = kmeans.labels_


# # green_label = np.argmax([np.mean(cluster_colors[np.where(kmeans_labels == i)], axis=0)[1] for i in range(classes)])

# greener_labels = np.argsort([np.mean(cluster_colors[np.where(kmeans_labels == i)], axis=0)[1] for i in range(classes)])[::-1][:classes // 3]

# greener_indices = np.concatenate([np.where(kmeans_labels == i)[0] for i in greener_labels])
# non_greener_indices = np.concatenate([np.where(kmeans_labels == i)[0] for i in range(classes) if i not in greener_labels])

# # visualize the clustered point cloud
# # for i in range(classes):
# o3d.visualization.draw_geometries([cluster_pcd.select_by_index(greener_indices)], window_name="Clustered Point Cloud")
# o3d.visualization.draw_geometries([cluster_pcd.select_by_index(non_greener_indices)], window_name="Clustered Point Cloud")





# # o3d.visualization.draw_geometries([cluster_pcd], window_name="Clustered Point Cloud")



[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


In [5]:
pcd = o3d.io.read_point_cloud("aligned_tree.pcd")
points = np.asarray(pcd.points)
colors = np.asarray(pcd.colors)

# Step 1: Determine Vertical Range
z_coords = points[:, 2]  # Assuming the third column represents the z-coordinate
min_z, max_z = np.min(z_coords), np.max(z_coords)

# Determine vertical range and create bins for layers
num_bins = 15  # Adjust the number of bins as needed for granularity
z_bins = np.linspace(min_z, max_z, num=num_bins+1)

# Calculate overall target brightness (you might adjust this strategy)
target_brightness = np.mean(calculate_brightness(colors))

# Apply adjustment
adjusted_colors = adjust_layer_brightness(colors, target_brightness, z_coords, z_bins)

# Replace original colors with adjusted colors for further processing
colors = adjusted_colors

# Filter points by darkness
filtered_indices = filter_by_darkness(colors, 5)
filtered_points = points[filtered_indices]
filtered_colors = colors[filtered_indices]


# remove outliers

# Calculate target brightness
target_brightness = np.mean(np.mean(filtered_colors, axis=0))

# Adjust brightness
adjusted_colors = adjust_brightness(filtered_colors, target_brightness)
# Step 1 & 2: Standardize Features and Combine with Emphasis on Color
scaler = StandardScaler()
scaled_points = scaler.fit_transform(filtered_points)  # Standardize positions
scaled_colors = scaler.fit_transform(adjusted_colors) * 2  # Standardize colors and weigh more heavily

combined_features = np.hstack((scaled_points, scaled_colors))  # Combine standardized features

# Step 3: DBSCAN Clustering on Combined Features
dbscan = DBSCAN(eps=0.28, min_samples=10).fit(combined_features)  # Adjust eps and min_samples as needed
labels = dbscan.labels_

# Identify the number of clusters (ignoring noise points, if any)
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
print(f"Estimated number of clusters: {n_clusters_}")

# Step 4: Visualization
# Create a copy of the original point cloud for visualization
viz_pcd = o3d.geometry.PointCloud()
viz_pcd.points = o3d.utility.Vector3dVector(points)  # Use original points
viz_pcd.colors = o3d.utility.Vector3dVector(colors)  # Use original colors initially

# Assign colors based on DBSCAN labels
unique_labels = set(labels)
# Create a new colors array for the original point cloud with default color
new_colors = np.array(viz_pcd.colors)

# Prepare a colormap
colors_per_cluster = plt.cm.get_cmap("hsv", len(unique_labels) + 1)(range(len(unique_labels) + 1))

# Apply colors based on labels to the filtered points only
for k, col in zip(unique_labels, colors_per_cluster):
    if k == -1:  # Noise
        col = [0, 0, 0, 1]  # Black for noise
    # Use filtered indices to find the positions in the original point cloud that correspond to each label
    match_indices = np.where(labels == k)[0]
    if len(match_indices) > 100:
        new_colors[filtered_indices[match_indices], :] = col[:3]

# Update the colors of the original point cloud for visualization
viz_pcd.colors = o3d.utility.Vector3dVector(new_colors)

# Visualization
o3d.visualization.draw_geometries([viz_pcd], window_name="Clustered Point Cloud with Original Data")

Estimated number of clusters: 108
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


In [6]:
pcd = o3d.io.read_point_cloud("branch_points.pcd")
points = np.asarray(pcd.points)
colors = np.asarray(pcd.colors)
print(len(points))

    # remove outliers left from the plane segmentation
clf = IsolationForest(contamination=0.5)
clf.fit(points)
y_pred = clf.predict(points)
outliers = np.where(y_pred == -1)
inliers = np.where(y_pred == 1)
points = np.delete(points, outliers, axis=0)
colors = np.delete(colors, outliers, axis=0)
print(len(points))

    # remove outliers left from the plane segmentation
pcd.points = o3d.utility.Vector3dVector(points)
pcd.colors = o3d.utility.Vector3dVector(colors)
pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
# visualize the point cloud
o3d.visualization.draw_geometries([pcd])


257045
128525
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


In [16]:
# Load and preprocess the point cloud
pcd = o3d.io.read_point_cloud("aligned_tree.pcd")
pcd = pcd.voxel_down_sample(voxel_size=0.005)

# dbscan to remove noise
labels = np.array(pcd.cluster_dbscan(eps=0.005, min_points=5, print_progress=True))
max_label = labels.max()
print(f"point cloud has {max_label + 1} clusters")

# Remove noise
points = np.asarray(pcd.points)
colors = np.asarray(pcd.colors)
inliers = np.where(labels != -1)
print(len(inliers[0]))
points = points[inliers]
colors = colors[inliers]
pcd.points = o3d.utility.Vector3dVector(points)
pcd.colors = o3d.utility.Vector3dVector(colors)

# Apply darkness and distance filters
darkness_percentile_threshold = 55  # Adjust based on preference
dark_points_indices = filter_by_darkness(np.asarray(pcd.colors), darkness_percentile_threshold)

layers = split_into_layers(pcd, num_layers=40)
distance_percentile_threshold = 60  # Adjust based on preference
distance_filtered_indices = filter_points_by_distance(layers, distance_percentile_threshold)

# Combine filters: only consider points passing both darkness and distance filters
combined_filtered_indices = np.intersect1d(dark_points_indices, distance_filtered_indices)

# Proceed with aggregate_covariances and subsequent steps on the filtered points
filtered_pcd = pcd.select_by_index(combined_filtered_indices)
aggregated_covariances, valid_indices = aggregate_covariances(filtered_pcd, radius=0.1, min_neighbors=20)
eigenvalues, eigenvectors = compute_eigenvalues_and_vectors_from_aggregated_covariances(aggregated_covariances)

# Detect bifurcation points on the filtered subset
bifurcation_indices = detect_bifurcation_points_with_layer_filtering(eigenvalues, valid_indices, layers, distance_percentile_threshold, covariance_percentile=5)

# Color a few nearest points to each bifurcation point red
find_and_color_neighbors(filtered_pcd, bifurcation_indices, num_neighbors=5)

# Visualize the filtered point cloud with highlighted neighbors of bifurcation points
o3d.visualization.draw_geometries([filtered_pcd], window_name="Filtered Point Cloud with Highlighted Neighbors")

Precompute neighbors.[========================================] 100%
point cloud has 3058 clusters=============>        ] 77%
62316


TypeError: detect_bifurcation_points_with_layer_filtering() got an unexpected keyword argument 'covariance_percentile'

In [8]:

# Load your processed point cloud
# pcd = o3d.io.read_point_cloud("3d-forest-classic-data/tree_1.pcd")
pcd = o3d.io.read_point_cloud("aligned_tree.pcd")

#downsample the point cloud
pcd = pcd.voxel_down_sample(voxel_size=0.01)

# dbscan to remove noise
labels = np.array(pcd.cluster_dbscan(eps=0.01, min_points=5, print_progress=True))
max_label = labels.max()
print(f"point cloud has {max_label + 1} clusters")

# Remove noise
points = np.asarray(pcd.points)
colors = np.asarray(pcd.colors)
inliers = np.where(labels != -1)
print(len(inliers[0]))
points = points[inliers]
colors = colors[inliers]
pcd.points = o3d.utility.Vector3dVector(points)
pcd.colors = o3d.utility.Vector3dVector(colors)


# Estimate normals
pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
# Estimate covariances
pcd.estimate_covariances(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
# Aggregate covariances
# Adjust your point cloud processing to include these modifications:
aggregated_covariances, valid_indices = aggregate_covariances(pcd, radius=0.1, min_neighbors=10)
eigenvalues, eigenvectors = compute_eigenvalues_and_vectors_from_aggregated_covariances(aggregated_covariances)

bifurcation_indices = detect_bifurcation_points(eigenvalues, valid_indices, percentile_threshold=5)

# Visualize bifurcation points (setting them as red)
colors = np.asarray(pcd.colors)
if len(colors) == 0:
    colors = np.zeros((len(pcd.points), 3))
colors[bifurcation_indices] = [1, 0, 0]  # Red
pcd.colors = o3d.utility.Vector3dVector(colors)

# Visualize the point cloud with bifurcation points highlighted
o3d.visualization.draw_geometries([pcd])

point cloud has 1016 clusters                                 ] 2%
21787
Precompute neighbors.[========================================] 100%
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


In [11]:
pcd = o3d.io.read_point_cloud("3d-forest-classic-data/tree_3.pcd")
points = np.asarray(pcd.points)

# detect bifurcation points
labels = np.array(pcd.cluster_dbscan(eps=0.105, min_points=10, print_progress=True))
max_label = labels.max()
print(f"point cloud has {max_label + 1} clusters")

# remove noise
points = np.asarray(pcd.points)
colors = np.asarray(pcd.colors)
inliers = np.where(labels != -1)
points = points[inliers]

# Estimate normals
pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
# Estimate covariances
pcd.estimate_covariances(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))

# visualize the point cloud
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points)
o3d.visualization.draw_geometries([pcd], window_name="Filtered Point Cloud")

# Aggregate covariances
aggregated_covariances, valid_indices = aggregate_covariances(pcd, radius=1, min_neighbors=5)
eigenvalues, eigenvectors = compute_eigenvalues_and_vectors_from_aggregated_covariances(aggregated_covariances)

bifurcation_indices = detect_bifurcation_points(eigenvalues, valid_indices, percentile_threshold=0)

print(len(bifurcation_indices))

# Visualize bifurcation points (setting them as red)
colors = np.zeros((len(pcd.points), 3))
colors = np.zeros((len(pcd.points), 3))
colors[bifurcation_indices] = [1, 0, 0]  # Red
pcd.colors = o3d.utility.Vector3dVector(colors)

# Visualize the point cloud with bifurcation points highlighted
o3d.visualization.draw_geometries([pcd])

Precompute neighbors.[===============>             point cloud has 165 clusters
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
Precompute neighbors.[========================================] 100%
0lustering[===========================>            ] 67%
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


In [14]:
# Load the preprocessed point cloud
pcd = o3d.io.read_point_cloud("branch_points.pcd")

# DBSCAN clustering
labels = np.array(pcd.cluster_dbscan(eps=0.004, min_points=5, print_progress=True))

# Find the label with the most points
counts = np.bincount(labels[labels >= 0])
largest_group_label = counts.argmax()

# Create a new point cloud with only the points from the largest group
points = np.asarray(pcd.points)
colors = np.asarray(pcd.colors)
largest_group_points = points[labels == largest_group_label]
largest_group_colors = colors[labels == largest_group_label]

largest_group_pcd = o3d.geometry.PointCloud()
largest_group_pcd.points = o3d.utility.Vector3dVector(largest_group_points)
largest_group_pcd.colors = o3d.utility.Vector3dVector(largest_group_colors)

#downsample the point cloud
largest_group_pcd = largest_group_pcd.voxel_down_sample(voxel_size=0.005)

# Estimate the normals for the largest group point cloud
largest_group_pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.05, max_nn=30))
# largest_group_pcd.orient_normals_consistent_tangent_plane(100)
largest_group_pcd.estimate_covariances(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.05, max_nn=30))

# Load your processed point cloud
pcd = largest_group_pcd
# Aggregate covariances
aggregated_covariances = aggregate_covariances(pcd)

# Compute eigenvalues and eigenvectors from aggregated covariances
eigenvalues, eigenvectors = compute_eigenvalues_and_vectors_from_aggregated_covariances(aggregated_covariances)

# Detect bifurcation points
bifurcation_indices = detect_bifurcation_points(eigenvalues, percentile=3)[0]

# Visualize bifurcation points (setting them as red)
colors = np.asarray(pcd.colors)
colors[bifurcation_indices] = [1, 0, 0]  # Red
pcd.colors = o3d.utility.Vector3dVector(colors)

# Visualize the point cloud with bifurcation points highlighted
o3d.visualization.draw_geometries([pcd])

Precompute neighbors.[========================================] 100%
[ 2.65006385 14.09401293  8.86118115 ...  6.36345604  5.46918539
  2.73384803]
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


In [5]:
pcd = o3d.io.read_point_cloud("cherry_1.pcd")

pcd.get_oriented_bounding_box()

# make frame
mesh_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.6, origin=[0, 0, 0])

# visualize the point cloud

o3d.visualization.draw_geometries([pcd, mesh_frame])

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


In [45]:
import open3d as o3d
import numpy as np

# Load the preprocessed point cloud
for i in range(1, 5):
    pcd = o3d.io.read_point_cloud(f"3d-forest-classic-data/tree_{i}.pcd")


    # DBSCAN clustering
    labels = np.array(pcd.cluster_dbscan(eps=0.1, min_points=5, print_progress=True))

    # Visualize the clustering result
    max_label = labels.max()
    colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))
    colors[labels < 0] = 0.5  # Set noise to black
    pcd.colors = o3d.utility.Vector3dVector(colors[:, :3])
    o3d.visualization.draw_geometries([pcd])


Precompute neighbors.[========================================] 100%
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
Precompute neighbors.[========================================] 100%
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
Precompute neighbors.[========================================] 100%
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
Precompute neighbors.[========================================] 100%
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


In [2]:
pcd_path = 'nonground_cherry_1.pcd'
pcd = o3d.io.read_point_cloud(pcd_path)

frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.5, origin=[0, 0, 0])

# o3d.visualization.draw_geometries([pcd, frame])

NameError: name 'o3d' is not defined

In [84]:
pcd_path = 'trunk_points.pcd'
pcd = o3d.io.read_point_cloud(pcd_path)

# Estimate the normals
pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))

# Visualize the point cloud with the normal estimates
o3d.visualization.draw_geometries([pcd], point_show_normal=True)

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


In [ ]:
alpha = 0.01
print(f"alpha={alpha:.3f}")
mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_alpha_shape(pcd, alpha)
mesh.compute_vertex_normals()
o3d.visualization.draw_geometries([mesh], mesh_show_back_face=True)

alpha=0.001
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


In [40]:
def create_rotation_matrix_from_axis_angle(axis, angle):
    # Ensure the axis is a unit vector
    axis = axis / np.linalg.norm(axis)
    
    # Create the skew-symmetric cross-product matrix of the axis
    ux, uy, uz = axis
    skew_sym_matrix = np.array([
        [0, -uz, uy],
        [uz, 0, -ux],
        [-uy, ux, 0]
    ])
    
    # Calculate the rotation matrix using Rodrigues' formula
    cos_theta = np.cos(angle)
    sin_theta = np.sin(angle)
    rotation_matrix = cos_theta * np.eye(3) + sin_theta * skew_sym_matrix + (1 - cos_theta) * np.outer(axis, axis)
    
    return rotation_matrix

In [58]:

def align_tree(pcd_path, output_path, center_threshold, normalize_colors=True, remove_points=True, distance_threshold=1, ransac_n=3, num_iterations=1000, frame_size=0.5):
    # load point cloud
    pcd = o3d.io.read_point_cloud(pcd_path)

    # downsample the point cloud

    if normalize_colors:
        # Normalize the colors
        pcd_colors = np.asarray(pcd.colors)
        pcd_colors /= 255.0
        pcd.colors = o3d.utility.Vector3dVector(pcd_colors)

    # Assuming filtered_pcd is your point cloud after filtering
    plane_model, inliers = pcd.segment_plane(distance_threshold=distance_threshold, ransac_n=ransac_n, num_iterations=num_iterations)
    [a, b, c, d] = plane_model

    for i in inliers:
        pcd.colors[i] = [1, 0, 0] # Red color

    # The ground plane normal vector
    ground_normal = np.array([a, b, c])

    # Normalize the ground normal vector
    ground_normal_normalized = ground_normal / np.linalg.norm(ground_normal)

    # Calculate the rotation needed
    # Z-axis unit vector
    z_axis = np.array([0, 0, 1])



    # Axis of rotation (cross product between ground normal and z-axis)
    rotation_axis = np.cross(z_axis, ground_normal_normalized)

    # Angle between ground normal and z-axis
    rotation_angle = np.arccos(np.clip(np.dot(z_axis, ground_normal_normalized), -1.0, 1.0))

    # Create the rotation matrix
    rotation_matrix = create_rotation_matrix_from_axis_angle(rotation_axis, rotation_angle)

    # Apply the rotation to align the ground plane with the XY plane
    rotated_points = np.dot(np.asarray(pcd.points), rotation_matrix)

    # flip across xy plane
    # rotated_points[:, 2] = -rotated_points[:, 2] 

    # Update the point cloud with the rotated points
    rotated_pcd = o3d.geometry.PointCloud()
    rotated_pcd.points = o3d.utility.Vector3dVector(rotated_points)
    rotated_pcd_points = np.asarray(rotated_pcd.points)
    rotated_pcd.colors = pcd.colors
    rotated_pcd_colors = np.asarray(rotated_pcd.colors)

    # Calculate the mean of the inlier points to translate the point cloud
    inlier_points = np.asarray(rotated_pcd.select_by_index(inliers).points)


    mean_z_of_inliers = np.mean(inlier_points, axis=0)[2]


    # Translate the point cloud down by the mean z value of the inliers
    translated_points = np.asarray(rotated_pcd.points) - np.array([0, 0, mean_z_of_inliers])
    inlier_points[:, 2] -= mean_z_of_inliers


    trunk_indices = np.where((translated_points[:, 2] < center_threshold) & (translated_points[:, 2] > np.max(inlier_points[:, 2])))

    # if there are no trunk points, use the inliers
    if len(trunk_indices[0]) == 0:
        trunk_indices = inliers

    # remove outliers from trunk points

    # rotated_pcd_colors[trunk_indices] = [0, 1, 0]


    mean_x_of_trunk = np.mean(rotated_pcd_points[trunk_indices], axis=0)[0]
    mean_y_of_trunk = np.mean(rotated_pcd_points[trunk_indices], axis=0)[1]

    translated_points[:, 0] -= mean_x_of_trunk
    translated_points[:, 1] -= mean_y_of_trunk


    rotated_pcd.colors = o3d.utility.Vector3dVector(rotated_pcd_colors)

    # remove the inliers
    translated_points = np.delete(translated_points, inliers, axis=0)
    rotated_pcd_colors = np.delete(rotated_pcd_colors, inliers, axis=0)

    trunk_indices = np.where((translated_points[:, 2] < center_threshold) & (translated_points[:, 2] > np.max(inlier_points[:, 2])))

    if len(trunk_indices[0]) != 0:

        # # remove the trunk points
        translated_points = np.delete(translated_points, trunk_indices, axis=0)
        rotated_pcd_colors = np.delete(rotated_pcd_colors, trunk_indices, axis=0)

    
    translated_pcd = o3d.geometry.PointCloud()
    translated_pcd.points = o3d.utility.Vector3dVector(translated_points)

    # Explicitly set colors for translated_pcd before visualization
    translated_pcd.colors = o3d.utility.Vector3dVector(rotated_pcd_colors)

    # Access the points from the translated and aligned point cloud
    # translated_points_array = np.asarray(translated_pcd.points)
    translated_colors_array = np.asarray(translated_pcd.colors)

    # remove outliers left from the plane segmentation
    clf = IsolationForest(contamination=0.01)
    clf.fit(translated_points)
    y_pred = clf.predict(translated_points)
    outliers = np.where(y_pred == -1)
    inliers = np.where(y_pred == 1)
    translated_points_array = np.delete(translated_points, outliers, axis=0)
    translated_colors_array = np.delete(translated_colors_array, outliers, axis=0)

    final_pcd = o3d.geometry.PointCloud()
    final_pcd.points = o3d.utility.Vector3dVector(translated_points_array)
    final_pcd.colors = o3d.utility.Vector3dVector(translated_colors_array)


    frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=frame_size, origin=[0, 0, 0])

    # downsample the point cloud
    # final_pcd = final_pcd.voxel_down_sample(voxel_size=0.1)

    # Visualize the point cloud
    o3d.visualization.draw_geometries([final_pcd, frame])

    # Save the translated and aligned point cloud
    o3d.io.write_point_cloud(output_path, final_pcd)




In [218]:
# pcd_path = 'trunk_points.pcd'
# translated_pcd = o3d.io.read_point_cloud(pcd_path)
# translated_pcd = translated_pcd.voxel_down_sample(voxel_size=0.005)
# translated_points = np.asarray(translated_pcd.points)
# sorted_indices = np.argsort(translated_points[:, 2])
# sorted_points = translated_points[sorted_indices]

# z_threshold = np.percentile(sorted_points[:, 2], 10)
# bottom_points = sorted_points[sorted_points[:, 2] <= z_threshold]

# # Find the bottom point closest to the origin
# distances_to_origin = np.linalg.norm(bottom_points, axis=1)
# index_closest_to_origin = np.argmin(distances_to_origin)
# starting_point = bottom_points[index_closest_to_origin]

# # Initialize the chain and visited points
# chain = [starting_point]
# visited = {tuple(starting_point)}

# # Define the search radii
# x_radius, y_radius, z_radius = 0.01, 0.01, 0.1  # Adjust as necessary

# # Iteratively extend the chain
# current_point = starting_point
# # i = 0
# while True:
#     nearest_point = find_nearest_unvisited_within_xyz_radius(current_point, translated_points, visited, x_radius, y_radius, z_radius)
#     if nearest_point is None:
#         nearest_point = find_nearest_unvisited_within_xyz_radius(current_point, translated_points, visited, x_radius * 3, y_radius * 3, z_radius * 3)
#         if nearest_point is None:
#             break
#         # break  # No further unvisited points within the specified radii
#     chain.append(nearest_point)
#     visited.add(tuple(nearest_point))
#     current_point = nearest_point
#     # print(i)
#     # i += 1

# # Prepare for visualization
# lines = [[i, i+1] for i in range(len(chain)-1)]
# chain_points = np.array(chain)
# line_set = o3d.geometry.LineSet(
#     points=o3d.utility.Vector3dVector(chain_points),
#     lines=o3d.utility.Vector2iVector(lines)
# )
# line_set.colors = o3d.utility.Vector3dVector([[1, 0, 0] for i in range(len(lines))])  # Red lines for connections

# # Visualize the point cloud and the chain
# o3d.visualization.draw_geometries([translated_pcd, line_set], window_name="Aligned Point Cloud with Chain")

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


In [59]:
align_tree('cherry_2.xyzrgb', 'cherry_2_aligned_tree.pcd', remove_points=False, center_threshold=2, normalize_colors=True, distance_threshold=5, ransac_n=3, num_iterations=1000, frame_size=40)

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


: 

In [ ]:

# Assuming 'translated_pcd' is your modified point cloud ready for processing
translated_points = np.asarray(translated_pcd.points)
colors = np.asarray(translated_pcd.colors)

# Step 1: Filter bottom points. Here, let's use the lowest 10% of points based on z-value.
z_values = translated_points[:, 2]
z_threshold = np.percentile(z_values, 10)  # Adjust the percentile as needed
bottom_indices = np.where(z_values <= z_threshold)[0]
bottom_points = translated_points[bottom_indices]

# Step 2: Find nearest neighbors. For simplicity, this example uses a brute-force approach.
connections = []
for i, point in enumerate(bottom_points):
    distances = np.linalg.norm(translated_points - point, axis=1)
    distances[bottom_indices[i]] = np.inf  # Ignore the point itself
    nearest_index = np.argmin(distances)
    connections.append([point, translated_points[nearest_index]])

# Convert connections to line set for visualization
lines = [[i, i + len(bottom_points)] for i in range(len(bottom_points))]
colors = [[1, 0, 0] for i in range(len(lines))]  # Red lines
line_set = o3d.geometry.LineSet()
line_set.points = o3d.utility.Vector3dVector(np.vstack((bottom_points, translated_points[bottom_indices])))
line_set.lines = o3d.utility.Vector2iVector(lines)
line_set.colors = o3d.utility.Vector3dVector(colors)

# Step 4: Visualize
o3d.visualization.draw_geometries([translated_pcd, line_set])


In [30]:


# Load the point cloud
pcd_path = 'granny.xyzrgb'
pcd = o3d.io.read_point_cloud(pcd_path)

pcd.colors = o3d.utility.Vector3dVector(np.asarray(pcd.colors) / 255.0)

# Assuming filtered_pcd is your point cloud after filtering
plane_model, inliers = pcd.segment_plane(distance_threshold=1, ransac_n=3, num_iterations=1000)
[a, b, c, d] = plane_model

# The ground plane normal vector
ground_normal = np.array([a, b, c])

# Normalize the ground normal vector
ground_normal_normalized = ground_normal / np.linalg.norm(ground_normal)

# Calculate the rotation needed
# Z-axis unit vector
z_axis = np.array([0, 0, 1])


In [44]:


# Axis of rotation (cross product between ground normal and z-axis)
rotation_axis = np.cross(z_axis, ground_normal_normalized)

# Angle between ground normal and z-axis
rotation_angle = np.arccos(np.clip(np.dot(z_axis, ground_normal_normalized), -1.0, 1.0))

# Create the rotation matrix
rotation_matrix = create_rotation_matrix_from_axis_angle(rotation_axis, rotation_angle)

# Apply the rotation to align the ground plane with the XY plane
rotated_points = np.dot(np.asarray(pcd.points), rotation_matrix)

# Update the point cloud with the rotated points
rotated_pcd = o3d.geometry.PointCloud()
rotated_pcd.points = o3d.utility.Vector3dVector(rotated_points)
rotated_pcd.colors = pcd.colors

# Calculate the mean of the inlier points to translate the point cloud
inlier_points = np.asarray(rotated_pcd.select_by_index(inliers).points)
mean_z_of_inliers = np.mean(inlier_points, axis=0)[2]

# Translate the point cloud down by the mean z value of the inliers
translated_points = np.asarray(rotated_pcd.points) - np.array([0, 0, mean_z_of_inliers])
translated_pcd = o3d.geometry.PointCloud()
translated_pcd.points = o3d.utility.Vector3dVector(translated_points)
# Explicitly set colors for translated_pcd before visualization
translated_pcd.colors = o3d.utility.Vector3dVector(np.asarray(rotated_pcd.colors))
# Define your Z threshold
z_min = 3.5  # Example threshold, adjust as needed
z_max = 60  # Example threshold, adjust as needed

# Access the points from the translated and aligned point cloud
translated_points_array = np.asarray(translated_pcd.points)

# Find indices of points that are above the Z threshold
above_threshold_indices = np.where((translated_points_array[:, 2] > z_min) & (translated_points_array[:, 2] < z_max))[0]

# Select only the points that are above the threshold
filtered_points = translated_points_array[above_threshold_indices]

# Also filter the colors to match the filtered points
filtered_colors = np.asarray(translated_pcd.colors)[above_threshold_indices]

# Create a new point cloud for the filtered points
filtered_pcd = o3d.geometry.PointCloud()
filtered_pcd.points = o3d.utility.Vector3dVector(filtered_points)
filtered_pcd.colors = o3d.utility.Vector3dVector(filtered_colors)

# remove statistical outliers
cl, ind = filtered_pcd.remove_statistical_outlier(nb_neighbors=20, std_ratio=1.0)

# Create a frame to visualize the filtered point cloud
frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=40, origin=[0, 0, 0])

# Visualization of the filtered point cloud
o3d.visualization.draw_geometries([filtered_pcd, frame], window_name="Filtered Point Cloud")

# Optionally, save the filtered point cloud
o3d.io.write_point_cloud('granny_filtered_pcd.pcd', filtered_pcd)


[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


True

In [5]:

# Load the point cloud
pcd_path = 'cherry_2.xyzrgb'
pcd = o3d.io.read_point_cloud(pcd_path)

# Convert the RGB to 0-1 scale
pcd.colors = o3d.utility.Vector3dVector(np.asarray(pcd.colors) / 255.0)

# Visualization
# o3d.visualization.draw_geometries([pcd])

# Thresholding points by Z value
z_threshold = -125
points = np.asarray(pcd.points)
colors = np.asarray(pcd.colors)

# Use boolean indexing to filter out points below the threshold
above_threshold_indices = points[:, 2] > z_threshold
filtered_points = points[above_threshold_indices]
filtered_colors = colors[above_threshold_indices]

# Create a new point cloud for the filtered data
filtered_pcd = o3d.geometry.PointCloud()
filtered_pcd.points = o3d.utility.Vector3dVector(filtered_points)
filtered_pcd.colors = o3d.utility.Vector3dVector(filtered_colors)

# remove outliers
plane_model, inliers = filtered_pcd.segment_plane(distance_threshold=0.06, ransac_n=3, num_iterations=1000)
filtered_pcd = filtered_pcd.select_by_index(inliers, invert=True)
filtered_pcd, ind = filtered_pcd.remove_statistical_outlier(nb_neighbors=20, std_ratio=1)

# Visualization of the filtered point cloud
o3d.visualization.draw_geometries([filtered_pcd], window_name="Filtered Point Cloud")

# Optionally save the filtered point cloud
# o3d.io.write_point_cloud('cherry2_filtered_pcd.pcd', filtered_pcd)


[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


In [3]:
# # Load the point cloud
# pcd_path = 'cherry2_filtered_pcd.pcd'
# pcd = o3d.io.read_point_cloud(pcd_path)

# # downsample the point cloud
# downpcd = pcd.voxel_down_sample(voxel_size=0.1)

# points = np.asarray(downpcd.points)

# rgb_values = np.asarray(downpcd.colors)

# print(points[:10])
# print(rgb_values[:10])
# # Scale spatial coordinates (X, Y, Z) and RGB values separately
# spatial_weight = 1  # Keep spatial weight as 1 for now
# color_weight = 10  # Increase this to give more weight to RGB values


# # Assuming 'points' is your Nx3 array of spatial coordinates
# # and 'rgb_values' is your Nx3 array of RGB values
# weighted_features = np.hstack((points * spatial_weight, rgb_values * color_weight))

# # Scale features to treat spatial and color dimensions equally
# scaler = StandardScaler()
# scaled_features = scaler.fit_transform(weighted_features)

# # Apply DBSCAN on scaled features
# dbscan = DBSCAN(eps=0.15, min_samples=10, algorithm='auto')  # Adjust eps and min_samples as needed
# labels = dbscan.fit_predict(scaled_features)

# # Assume 'points' and 'labels' are already defined from your DBSCAN clustering

# # Generate a color map, one color per cluster
# unique_labels = np.unique(labels)
# colors = plt.cm.rainbow(np.linspace(0, 1, len(unique_labels)))[:, :3]  # Exclude alpha channel

# # Assign colors to each point based on its cluster label
# point_colors = np.array([colors[label] if label != -1 else [0, 0, 0] for label in labels])

# # Create a new Open3D point cloud
# colored_pcd = o3d.geometry.PointCloud()
# colored_pcd.points = o3d.utility.Vector3dVector(points)
# colored_pcd.colors = o3d.utility.Vector3dVector(point_colors)

# # Visualization
# o3d.visualization.draw_geometries([colored_pcd], window_name="Colored Point Cloud")

[[ -24.54646492   20.71881676 -106.47665405]
 [ -85.41661072    0.25073901  -81.34545135]
 [ 104.64865875  -50.51711273  -49.45478821]
 [ -51.49118805   18.435812    -21.74301147]
 [ 136.69444275  -66.10697174  -74.6097641 ]
 [ -64.09218597   33.89157867  -24.89015579]
 [ 113.45696259  -31.25985336  -46.72667313]
 [ -77.57778168  -13.39559078  -22.51787758]
 [  94.6908493   -65.46487427  -16.20495224]
 [ -50.74226761  -61.21253967   -6.466043  ]]
[[0.40784314 0.40784314 0.31372549]
 [0.44313725 0.39215686 0.35686275]
 [0.45098039 0.43137255 0.43921569]
 [0.42745098 0.42352941 0.34901961]
 [0.41960784 0.40784314 0.39215686]
 [0.62745098 0.60784314 0.56862745]
 [0.4745098  0.45882353 0.45882353]
 [0.41960784 0.38039216 0.38431373]
 [0.52941176 0.52156863 0.50980392]
 [0.45490196 0.43137255 0.41568627]]
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


In [49]:
n_classes = 3

# ply_point_cloud = o3d.data.PLYPointCloud()
# pcd = o3d.io.read_point_cloud('nonground_cherry_1.pcd')

# # visualization
# # o3d.visualization.draw_geometries([pcd])

# # Load the point cloud data
# pcd = o3d.io.read_point_cloud('cherry_1.pcd')
# # downpcd = pcd.voxel_down_sample(voxel_size=0.005)

# Load the point cloud
pcd_path = 'cherry2_filtered_pcd.pcd'
pcd = o3d.io.read_point_cloud(pcd_path)

# downsample the point cloud
# pcd = pcd.voxel_down_sample(voxel_size=0.01)


# # remove outliers
# pcd, ind = pcd.remove_statistical_outlier(nb_neighbors=20, std_ratio=0.5)


# visualization
# o3d.visualization.draw_geometries([nonground_pcd])

# Save the non-ground point cloud for further processing
# o3d.io.write_point_cloud("nonground_cherry_1.pcd", nonground_pcd)




# Adjust brightness of the point cloud
def adjust_brightness(colors, target_brightness):
    current_brightness = np.mean(colors, axis=1)
    brightness_factor = target_brightness / (current_brightness + 1e-3)
    adjusted_colors = colors * brightness_factor[:, np.newaxis]
    return np.clip(adjusted_colors, 0, 1)



# remove outliers

# Calculate target brightness
target_brightness = np.mean(np.mean(np.asarray(pcd.colors), axis=1))

# Adjust brightness
adjusted_colors = adjust_brightness(np.asarray(pcd.colors), target_brightness)

# Apply K-Means clustering
kmeans = KMeans(n_clusters=n_classes, random_state=42).fit(adjusted_colors)
labels = kmeans.labels_

# Determine the greenest cluster
green_label = np.argmax([np.mean(adjusted_colors[labels == i], axis=0)[1] for i in range(n_classes)])

# Set non-green points to black
for i, label in enumerate(labels):
    if label != green_label:
        adjusted_colors[i] = [0, 0, 0]

# Update point cloud colors
pcd.colors = o3d.utility.Vector3dVector(adjusted_colors)

# Visualize the processed point cloud
o3d.visualization.draw_geometries([pcd], window_name="Processed Point Cloud 1")

# Apply a slight outlier removal on the green points
# Extract green points and their colors for outlier analysis
green_points = np.asarray(pcd.points)[labels == green_label]
green_colors = adjusted_colors[labels == green_label]

# Use IsolationForest for outlier detection to slightly remove browner colors
iso_forest = IsolationForest(contamination=0.31)  # Adjust this to control the degree of outlier removal
outliers = iso_forest.fit_predict(green_colors)



# # Update colors to remove slight outliers (set them to black)
for i, outlier in enumerate(outliers):
    if outlier == -1:
        green_colors[i] = [0, 0, 0]  # Set outlier color to black


# remove outliers
green_points = green_points[outliers != -1]
green_colors = green_colors[outliers != -1]

pcd.points = o3d.utility.Vector3dVector(green_points)
pcd.colors = o3d.utility.Vector3dVector(green_colors)
        

# Update the original point cloud with the new colors
adjusted_colors = np.asarray(pcd.colors)
pcd.colors = o3d.utility.Vector3dVector(adjusted_colors)

# Visualize and save the processed point cloud
o3d.visualization.draw_geometries([pcd], window_name="Processed Point Cloud")
o3d.io.write_point_cloud('cherry2_processed_pcd.pcd', pcd)





# First, identify points that retained their original color (not turned black)
original_color_indices = np.where(np.any(adjusted_colors != [0, 0, 0], axis=1))[0]

# Extract the colors of these points
original_color_colors = adjusted_colors[original_color_indices]

# Calculate average color and standard deviation for these points
average_color = np.mean(original_color_colors, axis=0)
std_dev_color = np.std(original_color_colors, axis=0)

print(f"average color = {average_color}, standard deviation = {std_dev_color}")

# Extract the points and colors of these points
original_color_points = np.asarray(pcd.points)[original_color_indices]
original_color_colors = adjusted_colors[original_color_indices]


/opt/homebrew/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
average color = [0.46986817 0.46475677 0.38066137], standard deviation = [0.01019852 0.0096533  0.00886173]


In [57]:

# Standardize the points
# scaler = StandardScaler()
# original_color_points = scaler.fit_transform(original_color_points)

# Apply DBSCAN to cluster these points
dbscan = DBSCAN(eps=0.6, min_samples=7)  # Tweak these parameters as needed
dbscan_labels = dbscan.fit_predict(original_color_points)

# Initialize variables to store analysis results
cluster_average_colors = []
cluster_std_dev_colors = []

# Analyze each cluster (excluding noise, labeled as -1)
for cluster_label in set(dbscan_labels):
    if cluster_label == -1:
        continue  # Skip noise
    cluster_indices = np.where(dbscan_labels == cluster_label)[0]
    cluster_colors = original_color_colors[cluster_indices]
    
    # Calculate average color and standard deviation for the cluster
    average_color = np.mean(cluster_colors, axis=0)
    std_dev_color = np.std(cluster_colors, axis=0)
    
    cluster_average_colors.append(average_color)
    cluster_std_dev_colors.append(std_dev_color)

# Print out the analysis results
num_clusters = len(cluster_average_colors)
print(f"Number of clusters with original color: {num_clusters}")
# for i, (avg_color, std_dev) in enumerate(zip(cluster_average_colors, cluster_std_dev_colors), start=1):
#     print(f"Cluster {i}: Average Color = {avg_color}, Standard Deviation = {std_dev}")

import matplotlib.pyplot as plt

# Generate a color map with a unique color for each cluster
color_map = plt.cm.get_cmap('jet', len(set(dbscan_labels)))

# Initialize an array to store the colors of the points
point_colors = np.zeros((len(original_color_points), 3))

# Assign each cluster a unique color
for cluster_label in set(dbscan_labels):
    if cluster_label == -1:
        continue  # Skip noise
    cluster_indices = np.where(dbscan_labels == cluster_label)[0]
    point_colors[cluster_indices] = color_map(cluster_label)[:3]  # Slice to keep only RGB, ignore alpha

# Create a new Open3D point cloud for visualization
clustered_pcd = o3d.geometry.PointCloud()
clustered_pcd.points = o3d.utility.Vector3dVector(original_color_points)
clustered_pcd.colors = o3d.utility.Vector3dVector(point_colors)

# Visualize the clustered point cloud
o3d.visualization.draw_geometries([clustered_pcd], window_name="Clustered Point Cloud")

# Initialize an array to store the colors of the points
original_point_colors = np.zeros((len(original_color_points), 3))

# Assign each point its original color
for i, label in enumerate(dbscan_labels):
    if label != -1:  # Skip noise
        original_point_colors[i] = original_color_colors[i]

# Create a new Open3D point cloud for visualization
original_color_pcd = o3d.geometry.PointCloud()
original_color_pcd.points = o3d.utility.Vector3dVector(original_color_points[dbscan_labels != -1])
original_color_pcd.colors = o3d.utility.Vector3dVector(original_point_colors[dbscan_labels != -1])

# Visualize the point cloud with original colors
o3d.visualization.draw_geometries([original_color_pcd], window_name="Original Color Point Cloud")

# Note: The eps parameter in DBSCAN affects the size and number of clusters
# It may need adjustment based on the scale of your data and the desired granularity of clustering



Number of clusters with original color: 1260
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


In [46]:
n_classes = 3

# ply_point_cloud = o3d.data.PLYPointCloud()
# pcd = o3d.io.read_point_cloud('nonground_cherry_1.pcd')

# # visualization
# # o3d.visualization.draw_geometries([pcd])

# # Load the point cloud data
# pcd = o3d.io.read_point_cloud('cherry_1.pcd')
# # downpcd = pcd.voxel_down_sample(voxel_size=0.005)

# Load the point cloud
pcd_path = 'granny_filtered_pcd.pcd'
pcd = o3d.io.read_point_cloud(pcd_path)

# downsample the point cloud
# pcd = pcd.voxel_down_sample(voxel_size=0.01)


# # remove outliers
# pcd, ind = pcd.remove_statistical_outlier(nb_neighbors=20, std_ratio=0.5)


# visualization
# o3d.visualization.draw_geometries([nonground_pcd])

# Save the non-ground point cloud for further processing
# o3d.io.write_point_cloud("nonground_cherry_1.pcd", nonground_pcd)




# Adjust brightness of the point cloud
def adjust_brightness(colors, target_brightness):
    current_brightness = np.mean(colors, axis=1)
    brightness_factor = target_brightness / (current_brightness + 1e-3)
    adjusted_colors = colors * brightness_factor[:, np.newaxis]
    return np.clip(adjusted_colors, 0, 1)



# remove outliers

# Calculate target brightness
target_brightness = np.mean(np.mean(np.asarray(pcd.colors), axis=1))

# Adjust brightness
adjusted_colors = adjust_brightness(np.asarray(pcd.colors), target_brightness)

# Apply K-Means clustering
kmeans = KMeans(n_clusters=n_classes, random_state=42).fit(adjusted_colors)
labels = kmeans.labels_

# Determine the greenest cluster
green_label = np.argmax([np.mean(adjusted_colors[labels == i], axis=0)[1] for i in range(n_classes)])

# Set non-green points to black
for i, label in enumerate(labels):
    if label != green_label:
        adjusted_colors[i] = [0, 0, 0]

# Update point cloud colors
pcd.colors = o3d.utility.Vector3dVector(adjusted_colors)

# Visualize the processed point cloud
o3d.visualization.draw_geometries([pcd], window_name="Processed Point Cloud 1")

# Apply a slight outlier removal on the green points
# Extract green points and their colors for outlier analysis
green_points = np.asarray(pcd.points)[labels == green_label]
green_colors = adjusted_colors[labels == green_label]

# Use IsolationForest for outlier detection to slightly remove browner colors
iso_forest = IsolationForest(contamination=0.31)  # Adjust this to control the degree of outlier removal
outliers = iso_forest.fit_predict(green_colors)



# # Update colors to remove slight outliers (set them to black)
for i, outlier in enumerate(outliers):
    if outlier == -1:
        green_colors[i] = [0, 0, 0]  # Set outlier color to black


# remove outliers
green_points = green_points[outliers != -1]
green_colors = green_colors[outliers != -1]

pcd.points = o3d.utility.Vector3dVector(green_points)
pcd.colors = o3d.utility.Vector3dVector(green_colors)
        

# Update the original point cloud with the new colors
adjusted_colors = np.asarray(pcd.colors)
pcd.colors = o3d.utility.Vector3dVector(adjusted_colors)

# Visualize and save the processed point cloud
o3d.visualization.draw_geometries([pcd], window_name="Processed Point Cloud")
o3d.io.write_point_cloud('cherry2_processed_pcd.pcd', pcd)





# First, identify points that retained their original color (not turned black)
original_color_indices = np.where(np.any(adjusted_colors != [0, 0, 0], axis=1))[0]

# Extract the colors of these points
original_color_colors = adjusted_colors[original_color_indices]

# Calculate average color and standard deviation for these points
average_color = np.mean(original_color_colors, axis=0)
std_dev_color = np.std(original_color_colors, axis=0)

print(f"average color = {average_color}, standard deviation = {std_dev_color}")

# Extract the points and colors of these points
original_color_points = np.asarray(pcd.points)[original_color_indices]
original_color_colors = adjusted_colors[original_color_indices]

# Standardize the points
scaler = StandardScaler()
original_color_points = scaler.fit_transform(original_color_points)

# Apply DBSCAN to cluster these points
dbscan = DBSCAN(eps=0.03, min_samples=7)  # Tweak these parameters as needed
dbscan_labels = dbscan.fit_predict(original_color_points)

# Initialize variables to store analysis results
cluster_average_colors = []
cluster_std_dev_colors = []

# Analyze each cluster (excluding noise, labeled as -1)
for cluster_label in set(dbscan_labels):
    if cluster_label == -1:
        continue  # Skip noise
    cluster_indices = np.where(dbscan_labels == cluster_label)[0]
    cluster_colors = original_color_colors[cluster_indices]
    
    # Calculate average color and standard deviation for the cluster
    average_color = np.mean(cluster_colors, axis=0)
    std_dev_color = np.std(cluster_colors, axis=0)
    
    cluster_average_colors.append(average_color)
    cluster_std_dev_colors.append(std_dev_color)

# Print out the analysis results
num_clusters = len(cluster_average_colors)
print(f"Number of clusters with original color: {num_clusters}")
# for i, (avg_color, std_dev) in enumerate(zip(cluster_average_colors, cluster_std_dev_colors), start=1):
#     print(f"Cluster {i}: Average Color = {avg_color}, Standard Deviation = {std_dev}")

import matplotlib.pyplot as plt

# Generate a color map with a unique color for each cluster
color_map = plt.cm.get_cmap('jet', len(set(dbscan_labels)))

# Initialize an array to store the colors of the points
point_colors = np.zeros((len(original_color_points), 3))

# Assign each cluster a unique color
for cluster_label in set(dbscan_labels):
    if cluster_label == -1:
        continue  # Skip noise
    cluster_indices = np.where(dbscan_labels == cluster_label)[0]
    point_colors[cluster_indices] = color_map(cluster_label)[:3]  # Slice to keep only RGB, ignore alpha

# Create a new Open3D point cloud for visualization
clustered_pcd = o3d.geometry.PointCloud()
clustered_pcd.points = o3d.utility.Vector3dVector(original_color_points)
clustered_pcd.colors = o3d.utility.Vector3dVector(point_colors)

# Visualize the clustered point cloud
o3d.visualization.draw_geometries([clustered_pcd], window_name="Clustered Point Cloud")

# Initialize an array to store the colors of the points
original_point_colors = np.zeros((len(original_color_points), 3))

# Assign each point its original color
for i, label in enumerate(dbscan_labels):
    if label != -1:  # Skip noise
        original_point_colors[i] = original_color_colors[i]

# Create a new Open3D point cloud for visualization
original_color_pcd = o3d.geometry.PointCloud()
original_color_pcd.points = o3d.utility.Vector3dVector(original_color_points[dbscan_labels != -1])
original_color_pcd.colors = o3d.utility.Vector3dVector(original_point_colors[dbscan_labels != -1])

# Visualize the point cloud with original colors
o3d.visualization.draw_geometries([original_color_pcd], window_name="Original Color Point Cloud")

# Note: The eps parameter in DBSCAN affects the size and number of clusters
# It may need adjustment based on the scale of your data and the desired granularity of clustering



/opt/homebrew/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
average color = [0.47976684 0.52175942 0.37214818], standard deviation = [0.0088597 0.0105764 0.0104489]
Number of clusters with original color: 1316
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


In [ ]:
n_classes = 3

# ply_point_cloud = o3d.data.PLYPointCloud()
# pcd = o3d.io.read_point_cloud('nonground_cherry_1.pcd')

# # visualization
# # o3d.visualization.draw_geometries([pcd])

# # Load the point cloud data
# pcd = o3d.io.read_point_cloud('cherry_1.pcd')
# # downpcd = pcd.voxel_down_sample(voxel_size=0.005)

# Load the point cloud
pcd_path = 'cherry2_filtered_pcd.pcd'
pcd = o3d.io.read_point_cloud(pcd_path)

# downsample the point cloud
# pcd = pcd.voxel_down_sample(voxel_size=0.01)


# # remove outliers
# pcd, ind = pcd.remove_statistical_outlier(nb_neighbors=20, std_ratio=0.5)


# visualization
# o3d.visualization.draw_geometries([nonground_pcd])

# Save the non-ground point cloud for further processing
# o3d.io.write_point_cloud("nonground_cherry_1.pcd", nonground_pcd)




# Adjust brightness of the point cloud
def adjust_brightness(colors, target_brightness):
    current_brightness = np.mean(colors, axis=1)
    brightness_factor = target_brightness / (current_brightness + 1e-3)
    adjusted_colors = colors * brightness_factor[:, np.newaxis]
    return np.clip(adjusted_colors, 0, 1)



# remove outliers

# Calculate target brightness
target_brightness = np.mean(np.mean(np.asarray(pcd.colors), axis=1))

# Adjust brightness
adjusted_colors = adjust_brightness(np.asarray(pcd.colors), target_brightness)

# Apply K-Means clustering
kmeans = KMeans(n_clusters=n_classes, random_state=42).fit(adjusted_colors)
labels = kmeans.labels_

# Determine the greenest cluster
green_label = np.argmax([np.mean(adjusted_colors[labels == i], axis=0)[1] for i in range(n_classes)])

# Set non-green points to black
for i, label in enumerate(labels):
    if label != green_label:
        adjusted_colors[i] = [0, 0, 0]

# Update point cloud colors
pcd.colors = o3d.utility.Vector3dVector(adjusted_colors)

# Visualize the processed point cloud
o3d.visualization.draw_geometries([pcd], window_name="Processed Point Cloud 1")

# Apply a slight outlier removal on the green points
# Extract green points and their colors for outlier analysis
green_points = np.asarray(pcd.points)[labels == green_label]
green_colors = adjusted_colors[labels == green_label]

# Use IsolationForest for outlier detection to slightly remove browner colors
iso_forest = IsolationForest(contamination=0.31)  # Adjust this to control the degree of outlier removal
outliers = iso_forest.fit_predict(green_colors)



# # Update colors to remove slight outliers (set them to black)
for i, outlier in enumerate(outliers):
    if outlier == -1:
        green_colors[i] = [0, 0, 0]  # Set outlier color to black


# remove outliers
green_points = green_points[outliers != -1]
green_colors = green_colors[outliers != -1]

pcd.points = o3d.utility.Vector3dVector(green_points)
pcd.colors = o3d.utility.Vector3dVector(green_colors)
        

# Update the original point cloud with the new colors
adjusted_colors = np.asarray(pcd.colors)
pcd.colors = o3d.utility.Vector3dVector(adjusted_colors)

# Visualize and save the processed point cloud
o3d.visualization.draw_geometries([pcd], window_name="Processed Point Cloud")
o3d.io.write_point_cloud('cherry2_processed_pcd.pcd', pcd)





# First, identify points that retained their original color (not turned black)
original_color_indices = np.where(np.any(adjusted_colors != [0, 0, 0], axis=1))[0]

# Extract the colors of these points
original_color_colors = adjusted_colors[original_color_indices]

# Calculate average color and standard deviation for these points
average_color = np.mean(original_color_colors, axis=0)
std_dev_color = np.std(original_color_colors, axis=0)

print(f"average color = {average_color}, standard deviation = {std_dev_color}")

# Extract the points and colors of these points
original_color_points = np.asarray(pcd.points)[original_color_indices]
original_color_colors = adjusted_colors[original_color_indices]

# Standardize the points
scaler = StandardScaler()
original_color_points = scaler.fit_transform(original_color_points)

# Apply DBSCAN to cluster these points
dbscan = DBSCAN(eps=0.03, min_samples=7)  # Tweak these parameters as needed
dbscan_labels = dbscan.fit_predict(original_color_points)

# Initialize variables to store analysis results
cluster_average_colors = []
cluster_std_dev_colors = []

# Analyze each cluster (excluding noise, labeled as -1)
for cluster_label in set(dbscan_labels):
    if cluster_label == -1:
        continue  # Skip noise
    cluster_indices = np.where(dbscan_labels == cluster_label)[0]
    cluster_colors = original_color_colors[cluster_indices]
    
    # Calculate average color and standard deviation for the cluster
    average_color = np.mean(cluster_colors, axis=0)
    std_dev_color = np.std(cluster_colors, axis=0)
    
    cluster_average_colors.append(average_color)
    cluster_std_dev_colors.append(std_dev_color)

# Print out the analysis results
num_clusters = len(cluster_average_colors)
print(f"Number of clusters with original color: {num_clusters}")
# for i, (avg_color, std_dev) in enumerate(zip(cluster_average_colors, cluster_std_dev_colors), start=1):
#     print(f"Cluster {i}: Average Color = {avg_color}, Standard Deviation = {std_dev}")

import matplotlib.pyplot as plt

# Generate a color map with a unique color for each cluster
color_map = plt.cm.get_cmap('jet', len(set(dbscan_labels)))

# Initialize an array to store the colors of the points
point_colors = np.zeros((len(original_color_points), 3))

# Assign each cluster a unique color
for cluster_label in set(dbscan_labels):
    if cluster_label == -1:
        continue  # Skip noise
    cluster_indices = np.where(dbscan_labels == cluster_label)[0]
    point_colors[cluster_indices] = color_map(cluster_label)[:3]  # Slice to keep only RGB, ignore alpha

# Create a new Open3D point cloud for visualization
clustered_pcd = o3d.geometry.PointCloud()
clustered_pcd.points = o3d.utility.Vector3dVector(original_color_points)
clustered_pcd.colors = o3d.utility.Vector3dVector(point_colors)

# Visualize the clustered point cloud
o3d.visualization.draw_geometries([clustered_pcd], window_name="Clustered Point Cloud")

# Initialize an array to store the colors of the points
original_point_colors = np.zeros((len(original_color_points), 3))

# Assign each point its original color
for i, label in enumerate(dbscan_labels):
    if label != -1:  # Skip noise
        original_point_colors[i] = original_color_colors[i]

# Create a new Open3D point cloud for visualization
original_color_pcd = o3d.geometry.PointCloud()
original_color_pcd.points = o3d.utility.Vector3dVector(original_color_points[dbscan_labels != -1])
original_color_pcd.colors = o3d.utility.Vector3dVector(original_point_colors[dbscan_labels != -1])

# Visualize the point cloud with original colors
o3d.visualization.draw_geometries([original_color_pcd], window_name="Original Color Point Cloud")

# Note: The eps parameter in DBSCAN affects the size and number of clusters
# It may need adjustment based on the scale of your data and the desired granularity of clustering



In [32]:
n_classes = 2

# ply_point_cloud = o3d.data.PLYPointCloud()
# pcd = o3d.io.read_point_cloud('nonground_cherry_1.pcd')

# # visualization
# # o3d.visualization.draw_geometries([pcd])

# # Load the point cloud data
# pcd = o3d.io.read_point_cloud('cherry_1.pcd')
# # downpcd = pcd.voxel_down_sample(voxel_size=0.005)

# Load the point cloud
pcd = o3d.io.read_point_cloud("aligned_tree.pcd")

# pcd = o3d.io.read_point_cloud("nonground_cherry_1.pcd")

# downsample the point cloud
# pcd = pcd.voxel_down_sample(voxel_size=0.01)


# # remove outliers
# pcd, ind = pcd.remove_statistical_outlier(nb_neighbors=20, std_ratio=0.5)


# visualization
o3d.visualization.draw_geometries([pcd])

# Save the non-ground point cloud for further processing
# o3d.io.write_point_cloud("nonground_cherry_1.pcd", nonground_pcd)




# Adjust brightness of the point cloud
def adjust_brightness(colors, target_brightness):
    current_brightness = np.mean(colors, axis=1)
    brightness_factor = target_brightness / (current_brightness + 1e-3)
    adjusted_colors = colors * brightness_factor[:, np.newaxis]
    return np.clip(adjusted_colors, 0, 1)



# remove outliers

# Calculate target brightness
target_brightness = np.mean(np.mean(np.asarray(pcd.colors), axis=1))

# Adjust brightness
adjusted_colors = adjust_brightness(np.asarray(pcd.colors), target_brightness)

# Apply K-Means clustering
kmeans = KMeans(n_clusters=n_classes, random_state=42).fit(adjusted_colors)
labels = kmeans.labels_

# Determine the greenest cluster
green_label = np.argmax([np.mean(adjusted_colors[labels == i], axis=0)[1] for i in range(n_classes)])

# Set non-green points to black
for i, label in enumerate(labels):
    if label == green_label:
        adjusted_colors[i] = [0, 0, 0]

# Update point cloud colors
pcd.colors = o3d.utility.Vector3dVector(adjusted_colors)

# Visualize the processed point cloud
o3d.visualization.draw_geometries([pcd], window_name="Processed Point Cloud 1")

# Apply a slight outlier removal on the green points
# Extract green points and their colors for outlier analysis
green_points = np.asarray(pcd.points)[labels == green_label]
green_colors = adjusted_colors[labels == green_label]

# Use IsolationForest for outlier detection to slightly remove browner colors
iso_forest = IsolationForest(contamination=0.4)  # Adjust this to control the degree of outlier removal
outliers = iso_forest.fit_predict(green_colors)



# # Update colors to remove slight outliers (set them to black)
for i, outlier in enumerate(outliers):
    if outlier == -1:
        green_colors[i] = [0, 0, 0]  # Set outlier color to black


# remove outliers
green_points = green_points[outliers != -1]
green_colors = green_colors[outliers != -1]

pcd.points = o3d.utility.Vector3dVector(green_points)
pcd.colors = o3d.utility.Vector3dVector(green_colors)
        

# Update the original point cloud with the new colors
adjusted_colors = np.asarray(pcd.colors)
pcd.colors = o3d.utility.Vector3dVector(adjusted_colors)

# Visualize and save the processed point cloud
o3d.visualization.draw_geometries([pcd], window_name="Processed Point Cloud")
o3d.io.write_point_cloud('processed_pcd.pcd', pcd)





# First, identify points that retained their original color (not turned black)
original_color_indices = np.where(np.any(adjusted_colors != [0, 0, 0], axis=1))[0]

# Extract the colors of these points
original_color_colors = adjusted_colors[original_color_indices]

# Calculate average color and standard deviation for these points
average_color = np.mean(original_color_colors, axis=0)
std_dev_color = np.std(original_color_colors, axis=0)

print(f"average color = {average_color}, standard deviation = {std_dev_color}")

# Extract the points and colors of these points
original_color_points = np.asarray(pcd.points)[original_color_indices]
original_color_colors = adjusted_colors[original_color_indices]

# Standardize the points
scaler = StandardScaler()
original_color_points = scaler.fit_transform(original_color_points)

# Apply DBSCAN to cluster these points
dbscan = DBSCAN(eps=0.03, min_samples=10)  # Tweak these parameters as needed
dbscan_labels = dbscan.fit_predict(original_color_points)

# Initialize variables to store analysis results
cluster_average_colors = []
cluster_std_dev_colors = []

# Analyze each cluster (excluding noise, labeled as -1)
for cluster_label in set(dbscan_labels):
    if cluster_label == -1:
        continue  # Skip noise
    cluster_indices = np.where(dbscan_labels == cluster_label)[0]
    cluster_colors = original_color_colors[cluster_indices]
    
    # Calculate average color and standard deviation for the cluster
    average_color = np.mean(cluster_colors, axis=0)
    std_dev_color = np.std(cluster_colors, axis=0)
    
    cluster_average_colors.append(average_color)
    cluster_std_dev_colors.append(std_dev_color)

# Print out the analysis results
num_clusters = len(cluster_average_colors)
print(f"Number of clusters with original color: {num_clusters}")
# for i, (avg_color, std_dev) in enumerate(zip(cluster_average_colors, cluster_std_dev_colors), start=1):
#     print(f"Cluster {i}: Average Color = {avg_color}, Standard Deviation = {std_dev}")

import matplotlib.pyplot as plt

# Generate a color map with a unique color for each cluster
color_map = plt.cm.get_cmap('jet', len(set(dbscan_labels)))

# Initialize an array to store the colors of the points
point_colors = np.zeros((len(original_color_points), 3))

# Assign each cluster a unique color
for cluster_label in set(dbscan_labels):
    if cluster_label == -1:
        continue  # Skip noise
    cluster_indices = np.where(dbscan_labels == cluster_label)[0]
    point_colors[cluster_indices] = color_map(cluster_label)[:3]  # Slice to keep only RGB, ignore alpha

# Create a new Open3D point cloud for visualization
clustered_pcd = o3d.geometry.PointCloud()
clustered_pcd.points = o3d.utility.Vector3dVector(original_color_points)
clustered_pcd.colors = o3d.utility.Vector3dVector(point_colors)

# Visualize the clustered point cloud
o3d.visualization.draw_geometries([clustered_pcd], window_name="Clustered Point Cloud")

# Initialize an array to store the colors of the points
original_point_colors = np.zeros((len(original_color_points), 3))

# Assign each point its original color
for i, label in enumerate(dbscan_labels):
    if label != -1:  # Skip noise
        original_point_colors[i] = original_color_colors[i]

# Create a new Open3D point cloud for visualization
original_color_pcd = o3d.geometry.PointCloud()
original_color_pcd.points = o3d.utility.Vector3dVector(original_color_points[dbscan_labels != -1])
original_color_pcd.colors = o3d.utility.Vector3dVector(original_point_colors[dbscan_labels != -1])

# Visualize the point cloud with original colors
o3d.visualization.draw_geometries([original_color_pcd], window_name="Original Color Point Cloud")

# Note: The eps parameter in DBSCAN affects the size and number of clusters
# It may need adjustment based on the scale of your data and the desired granularity of clustering



[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


/opt/homebrew/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
average color = [nan nan nan], standard deviation = [nan nan nan]


/opt/homebrew/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/homebrew/lib/python3.10/site-packages/numpy/core/_methods.py:181: RuntimeWarning: invalid value encountered in true_divide
  ret = um.true_divide(
/opt/homebrew/lib/python3.10/site-packages/numpy/core/_methods.py:264: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/opt/homebrew/lib/python3.10/site-packages/numpy/core/_methods.py:222: RuntimeWarning: invalid value encountered in true_divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
/opt/homebrew/lib/python3.10/site-packages/numpy/core/_methods.py:253: RuntimeWarning: invalid value encountered in true_divide
  ret = um.true_divide(


ValueError: Found array with 0 sample(s) (shape=(0, 3)) while a minimum of 1 is required by StandardScaler.

In [32]:
import numpy as np
import open3d as o3d

def adjust_brightness(colors, target_brightness):
    # Calculate current brightness as the mean of RGB values
    current_brightness = np.mean(colors, axis=1)
    
    # Calculate the factor needed to adjust each point to the target brightness
    brightness_factor = target_brightness / (current_brightness + 1e-6)
    
    # Adjust colors
    adjusted_colors = colors * brightness_factor[:, np.newaxis]
    
    # Ensure the adjusted colors are within valid range
    adjusted_colors = np.clip(adjusted_colors, 0, 1)
    
    return adjusted_colors

# Load the point cloud of blossom points
pcd_path = 'bloosom_points.pcd'
pcd = o3d.io.read_point_cloud(pcd_path)

# Extract colors from the loaded point cloud
all_colors = np.asarray(pcd.colors)

# Calculate the target brightness as the average brightness of all points
target_brightness = np.mean(np.mean(all_colors, axis=1))

# Adjust the brightness of all colors to the target brightness
normalized_colors = adjust_brightness(all_colors, target_brightness)

# Update the point cloud with normalized colors
pcd.colors = o3d.utility.Vector3dVector(normalized_colors)

# Save the adjusted point cloud back to a file or visualize it
adjusted_pcd_path = 'adjusted_blossom_points.pcd'
o3d.io.write_point_cloud(adjusted_pcd_path, pcd)
print(f"Adjusted blossom points saved to {adjusted_pcd_path}")

# Visualize the adjusted point cloud
o3d.visualization.draw_geometries([pcd], window_name="Adjusted Brightness Blossoms")

import numpy as np
import open3d as o3d
from sklearn.cluster import KMeans

# Load the brightness-adjusted point cloud
pcd_path = 'adjusted_blossom_points.pcd'
pcd = o3d.io.read_point_cloud(pcd_path)

# Extract colors and convert them to a suitable format for K-means
colors = np.asarray(pcd.colors)

# Apply K-means clustering to segment the points based on color
kmeans = KMeans(n_clusters=2, random_state=42).fit(colors)

# Use labels to determine color assignment
labels = kmeans.labels_

# Initialize an array for the new colors
new_colors = np.copy(colors)

# Calculate the greenness of each color
greenness = colors[:, 1] - (colors[:, 0] + colors[:, 2]) / 2

# Calculate the average greenness for each color class
average_greenness = [np.mean(greenness[labels == i]) for i in range(2)]

# Find the color class with the highest average greenness
greenest_class = np.argmax(average_greenness)

print(f"The greenest color class is {greenest_class}")

# Assign black color to one cluster and retain original color for the other
# Assuming we want to turn the first cluster (label=0) to black
for i, label in enumerate(labels):
    if label != greenest_class:
        new_colors[i] = [0, 0, 0]  # Set color to black for one class

# Update the point cloud colors
pcd.colors = o3d.utility.Vector3dVector(new_colors)

# Visualize the point cloud with one class as black and the other in original color
o3d.visualization.draw_geometries([pcd], window_name="KMeans Segmentation")

# Optionally, you can save this newly colored point cloud
o3d.io.write_point_cloud('kmeans_segmented_blossom_points.pcd', pcd)
print("KMeans segmented blossom points saved.")

from sklearn.cluster import DBSCAN

# First, identify points that retained their original color (not turned black)
original_color_indices = np.where(labels == greenest_class)[0]  # Assuming label=0 was assigned to black

# Extract the colors of these points
original_color_colors = colors[original_color_indices]

# Calculate average color and standard deviation for these points
average_color = np.mean(original_color_colors, axis=0)
std_dev_color = np.std(original_color_colors, axis=0)

print(f"average color = {average_color}, standard deviation = {std_dev_color}")

# Extract the points and colors of these points
original_color_points = np.asarray(pcd.points)[original_color_indices]
original_color_colors = colors[original_color_indices]

# Standardize the points
scaler = StandardScaler()
original_color_points = scaler.fit_transform(original_color_points)

# Apply DBSCAN to cluster these points
dbscan = DBSCAN(eps=0.025, min_samples=10)  # Tweak these parameters as needed
dbscan_labels = dbscan.fit_predict(original_color_points)

# Initialize variables to store analysis results
cluster_average_colors = []
cluster_std_dev_colors = []

# Analyze each cluster (excluding noise, labeled as -1)
for cluster_label in set(dbscan_labels):
    if cluster_label == -1:
        continue  # Skip noise
    cluster_indices = np.where(dbscan_labels == cluster_label)[0]
    cluster_colors = original_color_colors[cluster_indices]
    
    # Calculate average color and standard deviation for the cluster
    average_color = np.mean(cluster_colors, axis=0)
    std_dev_color = np.std(cluster_colors, axis=0)
    
    cluster_average_colors.append(average_color)
    cluster_std_dev_colors.append(std_dev_color)

# Print out the analysis results
num_clusters = len(cluster_average_colors)
print(f"Number of clusters with original color: {num_clusters}")
# for i, (avg_color, std_dev) in enumerate(zip(cluster_average_colors, cluster_std_dev_colors), start=1):
#     print(f"Cluster {i}: Average Color = {avg_color}, Standard Deviation = {std_dev}")

import matplotlib.pyplot as plt

# Generate a color map with a unique color for each cluster
color_map = plt.cm.get_cmap('jet', len(set(dbscan_labels)))

# Initialize an array to store the colors of the points
point_colors = np.zeros((len(original_color_points), 3))

# Assign each cluster a unique color
for cluster_label in set(dbscan_labels):
    if cluster_label == -1:
        continue  # Skip noise
    cluster_indices = np.where(dbscan_labels == cluster_label)[0]
    point_colors[cluster_indices] = color_map(cluster_label)[:3]  # Slice to keep only RGB, ignore alpha

# Create a new Open3D point cloud for visualization
clustered_pcd = o3d.geometry.PointCloud()
clustered_pcd.points = o3d.utility.Vector3dVector(original_color_points)
clustered_pcd.colors = o3d.utility.Vector3dVector(point_colors)

# Visualize the clustered point cloud
o3d.visualization.draw_geometries([clustered_pcd], window_name="Clustered Point Cloud")

# Initialize an array to store the colors of the points
original_point_colors = np.zeros((len(original_color_points), 3))

# Assign each point its original color
for i, label in enumerate(dbscan_labels):
    if label != -1:  # Skip noise
        original_point_colors[i] = original_color_colors[i]

# Create a new Open3D point cloud for visualization
original_color_pcd = o3d.geometry.PointCloud()
original_color_pcd.points = o3d.utility.Vector3dVector(original_color_points[dbscan_labels != -1])
original_color_pcd.colors = o3d.utility.Vector3dVector(original_point_colors[dbscan_labels != -1])

# Visualize the point cloud with original colors
o3d.visualization.draw_geometries([original_color_pcd], window_name="Original Color Point Cloud")

# Note: The eps parameter in DBSCAN affects the size and number of clusters
# It may need adjustment based on the scale of your data and the desired granularity of clustering


[Open3D WARNING] Read PCD failed: unable to open file: bloosom_points.pcd
[Open3D WARNING] Write PCD failed: unable to generate header.
Adjusted blossom points saved to adjusted_blossom_points.pcd
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] The number of points is 0 when creating axis-aligned bounding box.


/opt/homebrew/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/homebrew/lib/python3.10/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/opt/homebrew/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


The greenest color class is 0
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
KMeans segmented blossom points saved.
average color = [0.60650577 0.62670773 0.42736661], standard deviation = [0.02244739 0.03536474 0.06326522]
Number of clusters with original color: 885
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


In [ ]:
import numpy as np
import open3d as o3d

def adjust_brightness(colors, target_brightness):
    # Calculate current brightness as the mean of RGB values
    current_brightness = np.mean(colors, axis=1)
    
    # Calculate the factor needed to adjust each point to the target brightness
    brightness_factor = target_brightness / (current_brightness + 1e-6)
    
    # Adjust colors
    adjusted_colors = colors * brightness_factor[:, np.newaxis]
    
    # Ensure the adjusted colors are within valid range
    adjusted_colors = np.clip(adjusted_colors, 0, 1)
    
    return adjusted_colors

# Load the point cloud of blossom points
pcd_path = 'nonground_cherry_1.pcd'
pcd = o3d.io.read_point_cloud(pcd_path)

# Extract colors from the loaded point cloud
all_colors = np.asarray(pcd.colors)

# Calculate the target brightness as the average brightness of all points
target_brightness = np.mean(np.mean(all_colors, axis=1))

# Adjust the brightness of all colors to the target brightness
normalized_colors = adjust_brightness(all_colors, target_brightness)

# Update the point cloud with normalized colors
pcd.colors = o3d.utility.Vector3dVector(normalized_colors)

# Save the adjusted point cloud back to a file or visualize it
adjusted_pcd_path = 'adjusted_blossom_points.pcd'
o3d.io.write_point_cloud(adjusted_pcd_path, pcd)
print(f"Adjusted blossom points saved to {adjusted_pcd_path}")

# Visualize the adjusted point cloud
o3d.visualization.draw_geometries([pcd], window_name="Adjusted Brightness Blossoms")

import numpy as np
import open3d as o3d
from sklearn.cluster import KMeans

# Load the brightness-adjusted point cloud
pcd_path = 'adjusted_blossom_points.pcd'
pcd = o3d.io.read_point_cloud(pcd_path)

# Extract colors and convert them to a suitable format for K-means
colors = np.asarray(pcd.colors)

# Apply K-means clustering to segment the points based on color
kmeans = KMeans(n_clusters=2, random_state=42).fit(colors)

# Use labels to determine color assignment
labels = kmeans.labels_

# Initialize an array for the new colors
new_colors = np.copy(colors)

# Assign black color to one cluster and retain original color for the other
# Assuming we want to turn the first cluster (label=0) to black
for i, label in enumerate(labels):
    if label == 0:
        new_colors[i] = [0, 0, 0]  # Set color to black for one class

# Update the point cloud colors
pcd.colors = o3d.utility.Vector3dVector(new_colors)

# Visualize the point cloud with one class as black and the other in original color
o3d.visualization.draw_geometries([pcd], window_name="KMeans Segmentation")

# Optionally, you can save this newly colored point cloud
o3d.io.write_point_cloud('kmeans_segmented_blossom_points.pcd', pcd)
print("KMeans segmented blossom points saved.")

from sklearn.cluster import DBSCAN

# First, identify points that retained their original color (not turned black)
original_color_indices = np.where(labels != 0)[0]  # Assuming label=0 was assigned to black

# Extract the colors of these points
original_color_colors = colors[original_color_indices]

# Calculate average color and standard deviation for these points
average_color = np.mean(original_color_colors, axis=0)
std_dev_color = np.std(original_color_colors, axis=0)

print(f"average color = {average_color}, standard deviation = {std_dev_color}")

# Extract the points and colors of these points
original_color_points = np.asarray(pcd.points)[original_color_indices]
original_color_colors = colors[original_color_indices]

# Standardize the points
scaler = StandardScaler()
original_color_points = scaler.fit_transform(original_color_points)

# Apply DBSCAN to cluster these points
dbscan = DBSCAN(eps=0.025, min_samples=10)  # Tweak these parameters as needed
dbscan_labels = dbscan.fit_predict(original_color_points)

# Initialize variables to store analysis results
cluster_average_colors = []
cluster_std_dev_colors = []

# Analyze each cluster (excluding noise, labeled as -1)
for cluster_label in set(dbscan_labels):
    if cluster_label == -1:
        continue  # Skip noise
    cluster_indices = np.where(dbscan_labels == cluster_label)[0]
    cluster_colors = original_color_colors[cluster_indices]
    
    # Calculate average color and standard deviation for the cluster
    average_color = np.mean(cluster_colors, axis=0)
    std_dev_color = np.std(cluster_colors, axis=0)
    
    cluster_average_colors.append(average_color)
    cluster_std_dev_colors.append(std_dev_color)

# Print out the analysis results
num_clusters = len(cluster_average_colors)
print(f"Number of clusters with original color: {num_clusters}")
# for i, (avg_color, std_dev) in enumerate(zip(cluster_average_colors, cluster_std_dev_colors), start=1):
#     print(f"Cluster {i}: Average Color = {avg_color}, Standard Deviation = {std_dev}")

import matplotlib.pyplot as plt

# Generate a color map with a unique color for each cluster
color_map = plt.cm.get_cmap('jet', len(set(dbscan_labels)))

# Initialize an array to store the colors of the points
point_colors = np.zeros((len(original_color_points), 3))

# Assign each cluster a unique color
for cluster_label in set(dbscan_labels):
    if cluster_label == -1:
        continue  # Skip noise
    cluster_indices = np.where(dbscan_labels == cluster_label)[0]
    point_colors[cluster_indices] = color_map(cluster_label)[:3]  # Slice to keep only RGB, ignore alpha

# Create a new Open3D point cloud for visualization
clustered_pcd = o3d.geometry.PointCloud()
clustered_pcd.points = o3d.utility.Vector3dVector(original_color_points)
clustered_pcd.colors = o3d.utility.Vector3dVector(point_colors)

# Visualize the clustered point cloud
o3d.visualization.draw_geometries([clustered_pcd], window_name="Clustered Point Cloud")

# Initialize an array to store the colors of the points
original_point_colors = np.zeros((len(original_color_points), 3))

# Assign each point its original color
for i, label in enumerate(dbscan_labels):
    if label != -1:  # Skip noise
        original_point_colors[i] = original_color_colors[i]

# Create a new Open3D point cloud for visualization
original_color_pcd = o3d.geometry.PointCloud()
original_color_pcd.points = o3d.utility.Vector3dVector(original_color_points[dbscan_labels != -1])
original_color_pcd.colors = o3d.utility.Vector3dVector(original_point_colors[dbscan_labels != -1])

# Visualize the point cloud with original colors
o3d.visualization.draw_geometries([original_color_pcd], window_name="Original Color Point Cloud")

# Note: The eps parameter in DBSCAN affects the size and number of clusters
# It may need adjustment based on the scale of your data and the desired granularity of clustering


In [54]:

# # Load the point cloud data
# pcd = o3d.io.read_point_cloud('cherry_1.pcd')
# # downpcd = pcd.voxel_down_sample(voxel_size=0.005)

# # Perform plane segmentation to separate ground
# plane_model, inliers = pcd.segment_plane(distance_threshold=0.06,
#                                              ransac_n=3,
#                                              num_iterations=1000)

# # Extract inliers and outliers
# ground = pcd.select_by_index(inliers)
# nonground_pcd = pcd.select_by_index(inliers, invert=True)

# # Save the non-ground point cloud for further processing
# o3d.io.write_point_cloud("nonground_cherry_1.pcd", nonground_pcd)
ply_point_cloud = o3d.data.PLYPointCloud()
pcd = o3d.io.read_point_cloud('cherry2_filtered_pcd.pcd')
downpcd = pcd.voxel_down_sample(voxel_size=0.01)

points = np.asarray(downpcd.points)

rgb_values = np.asarray(downpcd.colors)

print(points[:10])
print(rgb_values[:10])
# Scale spatial coordinates (X, Y, Z) and RGB values separately
spatial_weight = 1  # Keep spatial weight as 1 for now
color_weight = 10  # Increase this to give more weight to RGB values
spherical_variance_threshold = 1
color_threshold = 1
# continuity_threshold = 0.5


# Assuming 'points' is your Nx3 array of spatial coordinates
# and 'rgb_values' is your Nx3 array of RGB values
weighted_features = np.hstack((points * spatial_weight, rgb_values * color_weight))

# Scale features to treat spatial and color dimensions equally
scaler = StandardScaler()
scaled_features = scaler.fit_transform(weighted_features)

# Apply DBSCAN on scaled features
dbscan = DBSCAN(eps=0.22, min_samples=10, algorithm='auto')  # Adjust eps and min_samples as needed
labels = dbscan.fit_predict(scaled_features)

# Assume 'points' and 'labels' are already defined from your DBSCAN clustering

# Generate a color map, one color per cluster
unique_labels = np.unique(labels)
colors = plt.cm.rainbow(np.linspace(0, 1, len(unique_labels)))[:, :3]  # Exclude alpha channel

# Assign colors to each point based on its cluster label
point_colors = np.array([colors[label] if label != -1 else [0, 0, 0] for label in labels])

# Create a new Open3D point cloud
colored_pcd = o3d.geometry.PointCloud()
colored_pcd.points = o3d.utility.Vector3dVector(points)
colored_pcd.colors = o3d.utility.Vector3dVector(point_colors)

# # Visualize the colored point cloud
# o3d.visualization.draw_geometries([colored_pcd], window_name='DBSCAN Clustering', width=3000, height=2000)


def is_spherical(cluster_points):
    centroid = np.mean(cluster_points, axis=0)
    distances = np.linalg.norm(cluster_points - centroid, axis=1)
    variance = np.var(distances)
    return variance < spherical_variance_threshold

def is_colorful(cluster_colors):
    # Calculate the standard deviation across the RGB values of the cluster
    color_std = np.std(cluster_colors, axis=0)
    return np.mean(color_std) > color_threshold

# Function to check cluster continuity
# def check_continuity(cluster_points):
#     # Compute pairwise distances within the cluster
#     pairwise_dist = pairwise_distances(cluster_points)
#     # Check if all points have at least one neighbor within the continuity threshold
#     return np.all(np.any(pairwise_dist < continuity_threshold, axis=1))

# Classify clusters
spherical_clusters = []
colorful_clusters = []
# Apply continuity check
# continuous_clusters = []

# for label in np.unique(labels):
#     if label == -1:  # Ignore noise
#         continue
#     cluster_points = points[labels == label]
#     if check_continuity(cluster_points):
#         continuous_clusters.append(label)

for label in np.unique(labels):
    if label == -1:  # Ignore noise
        continue
    cluster_points = points[labels == label]

    cluster_colors = rgb_values[labels == label] * 255  # Assuming rgb_values are [0, 1], scale back to [0, 255]
    
    # Check if the cluster is spherical
    if is_spherical(cluster_points):
        spherical_clusters.append(label)
    
    # Check if the cluster is colorful (non-grey)
    if is_colorful(cluster_colors):
        colorful_clusters.append(label)

# Assume a cluster is a blossom if it is both spherical and colorful
blossom_clusters = set(spherical_clusters)
# Color the clusters: Red for blossoms, Grey for non-blossom


# Original point cloud colors
original_colors = np.asarray(downpcd.colors)

# Initialize all points as red first (for non-blossoms)
# point_colors = np.ones_like(original_colors) * original_colors
# point_colors = np.ones_like(original_colors) * [1, 1, 1]
point_colors = np.ones_like(original_colors) * [1, 0, 0]


blossom_count = 0
# Assign the original color to points that are part of the blossom clusters
for i, label in enumerate(labels):
    if label in blossom_clusters:
        point_colors[i] = [0.5, 0.5, 0.5]
        # point_colors[i] = original_colors[i]

# Create a new Open3D point cloud with updated colors
colored_pcd.points = o3d.utility.Vector3dVector(points)
colored_pcd.colors = o3d.utility.Vector3dVector(point_colors)

# Visualize the updated point cloud with blossoms in original color and the rest in grey
o3d.visualization.draw_geometries([colored_pcd], window_name='Blossom Classification', width=3000, height=2000)


[[ -21.52998924 -100.98542023  -83.90379333]
 [ -51.49118805   18.435812    -21.74301147]
 [ 136.69444275  -66.10697174  -74.6097641 ]
 [-104.28185272  -20.62865448  -53.39678574]
 [ -78.24438477  -10.88687229  -88.60510254]
 [ -15.80795574 -141.78492737  -31.64421272]
 [ -40.37775421  -81.65332031  -84.59954071]
 [ -64.09218597   33.89157867  -24.89015579]
 [  94.08815765  -82.3712616   -25.92399788]
 [ -64.60190582   38.06379318  -96.91270447]]
[[0.4        0.4        0.3254902 ]
 [0.42745098 0.42352941 0.34901961]
 [0.41960784 0.40784314 0.39215686]
 [0.52941176 0.50588235 0.43921569]
 [0.36470588 0.35294118 0.34117647]
 [0.42352941 0.4        0.36470588]
 [0.48235294 0.43921569 0.41176471]
 [0.62745098 0.60784314 0.56862745]
 [0.5254902  0.52156863 0.55686275]
 [0.40392157 0.38823529 0.36078431]]


KeyboardInterrupt: 

In [6]:


ply_point_cloud = o3d.data.PLYPointCloud()
pcd = o3d.io.read_point_cloud('cherry_1.pcd')
# downpcd = pcd.voxel_down_sample(voxel_size=0.004)

points = np.asarray(pcd.points)

rgb_values = np.asarray(pcd.colors)
# Threshold for identifying green points
# Adjust these thresholds to correctly capture the green color of the blossoms in your dataset
green_threshold = 0.42  # Minimum green value to be considered green
red_blue_threshold = 0.4  # Maximum value for red and blue to ensure the point is predominantly green

# Identify green points
is_green = (rgb_values[:, 1] > green_threshold) & (rgb_values[:, 0] < red_blue_threshold) & (rgb_values[:, 2] < red_blue_threshold)
print(len(points))
green_points = points[is_green]
print(len(green_points))

# Apply DBSCAN to green points to identify individual blossoms
dbscan_green = DBSCAN(eps=0.01, min_samples=5)  # Adjust eps and min_samples based on your dataset
green_labels = dbscan_green.fit_predict(green_points)

# Count the number of blossoms as the number of unique clusters found, excluding noise (label = -1)
num_blossoms = len(set(green_labels)) - (1 if -1 in green_labels else 0)
print("Estimated number of blossoms:", num_blossoms)

# Generate a color map for the green clusters
unique_green_labels = np.unique(green_labels)
green_colors = plt.cm.spring(np.linspace(0, 1, len(unique_green_labels)))

# Get the original colors of the green points
original_green_colors = rgb_values[is_green]

# Create and visualize a point cloud for green clusters
green_pcd = o3d.geometry.PointCloud()
green_pcd.points = o3d.utility.Vector3dVector(green_points)
green_pcd.colors = o3d.utility.Vector3dVector(original_green_colors)

o3d.visualization.draw_geometries([green_pcd], window_name='Green Blossoms', width=3000, height=2000)
# # Assuming 'points' is your data and 'labels' are the labels obtained from DBSCAN
# unique_labels = np.unique(labels)
# colors = plt.cm.rainbow(np.linspace(0, 1, len(unique_labels)))


# # Load the point cloud data
# pcd = o3d.io.read_point_cloud('nonground_cherry_1.pcd')
# downpcd = pcd.voxel_down_sample(voxel_size=0.004)
# points = np.asarray(downpcd.points)

# # Scale spatial coordinates only, given we're focusing on structure
# scaler = StandardScaler()
# scaled_points = scaler.fit_transform(points)

# # Apply DBSCAN on scaled spatial features, adjusted for structure emphasis
# dbscan = DBSCAN(eps=0.1, min_samples=50)  # Adjust eps and min_samples based on visual inspection
# labels = dbscan.fit_predict(scaled_points)

# # Filter clusters based on size to eliminate noise and focus on main structural parts
# unique_labels, counts = np.unique(labels, return_counts=True)

# # Assume clusters representing structure have more points than a threshold
# structure_clusters = unique_labels[counts > 100]  # Adjust threshold based on data

# # Create a mask for points belonging to the structural clusters
# structure_mask = np.isin(labels, structure_clusters)

# # Extract structure points
# structure_points = points[structure_mask]

# # Create a point cloud for visualization
# structure_pcd = o3d.geometry.PointCloud()
# structure_pcd.points = o3d.utility.Vector3dVector(structure_points)
# # Visualize the structural skeleton
# o3d.visualization.draw_geometries([structure_pcd], window_name="Tree Structure", width=3000, height=2000)



1000223
8342
Estimated number of blossoms: 140
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


In [ ]:

# import numpy as np
# import open3d as o3d
# from sklearn.cluster import AgglomerativeClustering
# from sklearn.preprocessing import StandardScaler

# # Load and downsample the point cloud
# pcd = o3d.io.read_point_cloud('nonground_cherry_1.pcd')
# downpcd = pcd.voxel_down_sample(voxel_size=0.01)
# points = np.asarray(downpcd.points)
# rgb_values = np.asarray(downpcd.colors)

# # Feature Engineering: Combine spatial and RGB color features
# # Note: Color weights might be adjusted to emphasize the grey shade similarity
# # features = np.hstack((points, rgb_values * 10))  # Adjust color weight as needed

# # # Scale combined features
# # scaler = StandardScaler()
# # scaled_features = scaler.fit_transform(features)

# # Apply Agglomerative Clustering
# # The choice of 'distance_threshold' allows us to control the granularity of the clustering
# agg_clust = AgglomerativeClustering(n_clusters=None, distance_threshold=0.5, linkage='ward')  # Adjust the threshold
# labels = agg_clust.fit_predict(points)

# # # Identify significant clusters (with more than a threshold number of points)
# # unique_labels, counts = np.unique(labels, return_counts=True)
# # significant_clusters = unique_labels[counts > 10]  # Threshold for significant clusters, adjust as needed

# # # Filter points belonging to significant clusters
# # significant_mask = np.isin(labels, significant_clusters)
# # significant_points = points[significant_mask]
# # significant_colors = rgb_values[significant_mask]  # For visual verification

# # # Create a point cloud for the significant clusters
# # significant_pcd = o3d.geometry.PointCloud()
# # significant_pcd.points = o3d.utility.Vector3dVector(significant_points)
# # significant_pcd.colors = o3d.utility.Vector3dVector(significant_colors)  # Use original colors for visualization

# # # Visualize the filtered structural components
# # o3d.visualization.draw_geometries([significant_pcd], window_name="Significant Tree Structure", width=3000, height=2000)


In [ ]:

# from sklearn.preprocessing import StandardScaler
# import matplotlib.pyplot as plt
# import numpy as np
# import open3d as o3d

# # Load and prepare the point cloud
# pcd = o3d.io.read_point_cloud('nonground_cherry_1.pcd')
# downpcd = pcd.voxel_down_sample(voxel_size=0.004)
# points = np.asarray(downpcd.points)
# rgb_values = np.asarray(downpcd.colors)

# # Combine spatial and color features with weighting
# features = np.hstack((points, rgb_values * 5))  # Adjust the weight for color features as needed

# # Scale the combined features
# scaler = StandardScaler()
# scaled_features = scaler.fit_transform(features)

# # Initialize and fit the Gaussian Mixture Model
# # Adjust 'n_components' based on the expected segmentation of the tree and its surroundings
# gmm = GaussianMixture(n_components=2, covariance_type='full', random_state=0)
# gmm_labels = gmm.fit_predict(scaled_features)

# # Generate a color map for the GMM labels
# unique_labels = np.unique(gmm_labels)
# colors = plt.cm.rainbow(np.linspace(0, 1, len(unique_labels)))

# # Assign colors to each point based on its GMM label
# gmm_point_colors = np.array([colors[label][:3] for label in gmm_labels])  # Exclude alpha channel

# # Create a new Open3D point cloud for the GMM result
# gmm_colored_pcd = o3d.geometry.PointCloud()
# gmm_colored_pcd.points = o3d.utility.Vector3dVector(points)
# gmm_colored_pcd.colors = o3d.utility.Vector3dVector(gmm_point_colors)

# # Visualize the GMM-segmented point cloud
# o3d.visualization.draw_geometries([gmm_colored_pcd], window_name='GMM Segmentation', width=3000, height=2000)


In [14]:

# import json
# # Load the point cloud
# pcd_path = 'cherry_1.pcd'  # Adjust path if necessary
# pcd = o3d.io.read_point_cloud(pcd_path)
# # pcd = pcd.voxel_down_sample(voxel_size=0.004)
# points = np.asarray(pcd.points)
# rgb_values = np.asarray(pcd.colors)

# # Define a mapping from category_id to color (normalized to [0, 1] range)
# category_to_color = {
#     1: [0, 113/255, 188/255],  # Noise
#     2: [216/255, 82/255, 24/255],  # Trunk
#     3: [236/255, 176/255, 31/255],  # Branch
#     4: [125/255, 46/255, 141/255],  # Leaf
#     5: [118/255, 171/255, 47/255],  # Blossom
#     6: [161/255, 19/255, 46/255]   # Ground
# }

# # load blossoms-v0.5.json
# with open('blossoms-v0.5.json') as f:
#     data = json.load(f)


# point_annotations = data['dataset']['samples'][0]['labels']['ground-truth']['attributes']['point_annotations']

# # Convert point annotations to a NumPy array for efficient processing
# point_annotations_np = np.array(point_annotations)

# # Filter to get the indices of points labeled as blossoms (label 4)
# blossom_indices = np.where(point_annotations_np == 1)[0]

# # Select the points and colors for blossoms
# blossom_points = np.asarray(pcd.points)[blossom_indices]
# blossom_colors = np.asarray(pcd.colors)[blossom_indices]

# # Create a new point cloud for the blossom points
# # blossom_pcd = o3d.geometry.PointCloud()
# # blossom_pcd.points = o3d.utility.Vector3dVector(blossom_points)
# # blossom_pcd.colors = o3d.utility.Vector3dVector(blossom_colors)

# # # Save the blossom point cloud to a new PCD file
# # blossom_pcd_path = 'trunk_points.pcd'
# # o3d.io.write_point_cloud(blossom_pcd_path, blossom_pcd)

# # print(f"Blossom points saved to {blossom_pcd_path}")



# scaler = StandardScaler()
# scaled_blossom_points = scaler.fit_transform(blossom_points)


# # Ensure there are blossom points before applying DBSCAN
# if len(blossom_points) > 0:
#     dbscan_blossoms = DBSCAN(eps=0.015, min_samples=25, n_jobs=-1, leaf_size=1)
#     blossom_labels = dbscan_blossoms.fit_predict(scaled_blossom_points)

#     # Count the number of distinct blossom clusters
#     num_blossoms = len(set(blossom_labels)) - (1 if -1 in blossom_labels else 0)
#     print("Estimated number of blossoms:", num_blossoms)

#     # For visualization, you may assign colors based on the clustering result
#     # Ensure blossom_colors is updated if you wish to use it for visualization
# else:
#     print("No blossom points were found.")


# # Prepare a color palette for clusters
# num_clusters = len(set(blossom_labels)) - (1 if -1 in blossom_labels else 0)
# cluster_colors = plt.cm.get_cmap('viridis', num_clusters)

# # Initialize an array to hold colors for all points in the original point cloud
# all_point_colors = np.array([category_to_color[5] for _ in range(len(points))])  # Start with all points as blossom color

# # Assign colors to blossom points based on DBSCAN cluster labels
# for i, cluster_label in enumerate(blossom_labels):
#     if cluster_label == -1:  # Noise
#         color = [0, 0, 0]  # Black for noise
#     else:
#         color = cluster_colors(cluster_label)[:3]  # Exclude alpha channel from RGBA
#     all_point_colors[blossom_indices[i]] = color  # Update color for this point

# # Update the colors of the original point cloud
# pcd.colors = o3d.utility.Vector3dVector(all_point_colors)

# # Now select indices where the label is 4 (leaves)
# blossom_indices = np.where(point_annotations_np == 1)[0]


# # Select the leaf points using these indices
# blossom_points = points[blossom_indices]
# blossom_colors = rgb_values[blossom_indices]  # If you want to use original colors for visualization

# # optional visualization of the blossom points
# # # Create a new point cloud for the blossom points with DBSCAN colors
# # dbscan_colored_pcd = o3d.geometry.PointCloud()
# # dbscan_colored_pcd.points = o3d.utility.Vector3dVector(blossom_points)
# # dbscan_colored_pcd.colors = o3d.utility.Vector3dVector(blossom_colors)

# # # Visualize the DBSCAN clustered blossom points
# # o3d.visualization.draw_geometries([dbscan_colored_pcd], window_name='DBSCAN Blossom Clustering', width=3000, height=2000)

# # Prepare a color palette for clusters, including a color for noise (-1 labels)
# unique_labels = np.unique(blossom_labels)
# num_unique_labels = len(unique_labels[unique_labels != -1])
# cluster_colors = plt.cm.get_cmap('viridis', num_unique_labels)

# # Map DBSCAN labels to colors
# label_to_color = {
#     label: cluster_colors(i)[:3] for i, label in enumerate(unique_labels)
# }

# # For noise points, you might want to use a distinct color, like black
# label_to_color[-1] = [0, 0, 0]

# # Assign colors to blossom points based on DBSCAN cluster labels
# dbscan_colors = np.array([label_to_color[label] for label in blossom_labels])

# # Create a new point cloud for the blossom points with DBSCAN colors
# dbscan_colored_pcd = o3d.geometry.PointCloud()
# dbscan_colored_pcd.points = o3d.utility.Vector3dVector(blossom_points)
# dbscan_colored_pcd.colors = o3d.utility.Vector3dVector(dbscan_colors)

# # Visualize the DBSCAN clustered blossom points
# o3d.visualization.draw_geometries([dbscan_colored_pcd], window_name='DBSCAN Blossom Clustering', width=3000, height=2000)


Blossom points saved to trunk_points.pcd
Estimated number of blossoms: 0
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


In [ ]:
# import numpy as np
# import open3d as o3d
# import json

# # Function to adjust brightness
# def adjust_brightness(colors, target_brightness):
#     # Calculate current brightness as the mean of RGB values
#     current_brightness = np.mean(colors, axis=1)
    
#     # Calculate the factor needed to adjust each point to the target brightness
#     brightness_factor = target_brightness / (current_brightness + 1e-6)
    
#     # Adjust colors
#     adjusted_colors = colors * brightness_factor[:, np.newaxis]
    
#     # Ensure the adjusted colors are within valid range
#     adjusted_colors = np.clip(adjusted_colors, 0, 1)
    
#     return adjusted_colors

# # Load the point cloud
# pcd_path = 'blossom_points.pcd'  # Adjust the path to your point cloud file
# pcd = o3d.io.read_point_cloud(pcd_path)

# # Extract all points and colors
# all_points = np.asarray(pcd.points)
# all_colors = np.asarray(pcd.colors)

# # Target brightness calculation and adjustment should only be applied to blossom points
# blossom_colors = all_colors[blossom_indices]
# target_brightness = np.mean(np.mean(blossom_colors, axis=1))
# normalized_colors = adjust_brightness(blossom_colors, target_brightness)

# # Create a full color array and update only the blossom indices with normalized colors
# full_normalized_colors = all_colors.copy()
# full_normalized_colors[blossom_indices] = normalized_colors

# # Update the point cloud with the full normalized colors
# pcd.colors = o3d.utility.Vector3dVector(full_normalized_colors)

# # Visualize the point cloud with brightness-normalized blossom colors
# o3d.visualization.draw_geometries([pcd], window_name="Brightness Normalized Blossoms")


In [3]:
# import numpy as np
# import open3d as o3d

# def adjust_brightness(colors, target_brightness):
#     # Calculate current brightness as the mean of RGB values
#     current_brightness = np.mean(colors, axis=1)
    
#     # Calculate the factor needed to adjust each point to the target brightness
#     brightness_factor = target_brightness / (current_brightness + 1e-6)
    
#     # Adjust colors
#     adjusted_colors = colors * brightness_factor[:, np.newaxis]
    
#     # Ensure the adjusted colors are within valid range
#     adjusted_colors = np.clip(adjusted_colors, 0, 1)
    
#     return adjusted_colors

# # Load the point cloud of blossom points
# pcd_path = 'blossom_points.pcd'
# pcd = o3d.io.read_point_cloud(pcd_path)

# # Extract colors from the loaded point cloud
# all_colors = np.asarray(pcd.colors)

# # Calculate the target brightness as the average brightness of all points
# target_brightness = np.mean(np.mean(all_colors, axis=1))

# # Adjust the brightness of all colors to the target brightness
# normalized_colors = adjust_brightness(all_colors, target_brightness)

# # Update the point cloud with normalized colors
# pcd.colors = o3d.utility.Vector3dVector(normalized_colors)

# # Save the adjusted point cloud back to a file or visualize it
# adjusted_pcd_path = 'adjusted_blossom_points.pcd'
# o3d.io.write_point_cloud(adjusted_pcd_path, pcd)
# print(f"Adjusted blossom points saved to {adjusted_pcd_path}")

# # Visualize the adjusted point cloud
# o3d.visualization.draw_geometries([pcd], window_name="Adjusted Brightness Blossoms")

# import numpy as np
# import open3d as o3d
# from sklearn.cluster import KMeans

# # Load the brightness-adjusted point cloud
# pcd_path = 'adjusted_blossom_points.pcd'
# pcd = o3d.io.read_point_cloud(pcd_path)

# # Extract colors and convert them to a suitable format for K-means
# colors = np.asarray(pcd.colors)

# # Apply K-means clustering to segment the points based on color
# kmeans = KMeans(n_clusters=2, random_state=42).fit(colors)

# # Use labels to determine color assignment
# labels = kmeans.labels_

# # Initialize an array for the new colors
# new_colors = np.copy(colors)

# # Assign black color to one cluster and retain original color for the other
# # Assuming we want to turn the first cluster (label=0) to black
# for i, label in enumerate(labels):
#     if label == 0:
#         new_colors[i] = [0, 0, 0]  # Set color to black for one class

# # Update the point cloud colors
# pcd.colors = o3d.utility.Vector3dVector(new_colors)

# # Visualize the point cloud with one class as black and the other in original color
# o3d.visualization.draw_geometries([pcd], window_name="KMeans Segmentation")

# # Optionally, you can save this newly colored point cloud
# o3d.io.write_point_cloud('kmeans_segmented_blossom_points.pcd', pcd)
# print("KMeans segmented blossom points saved.")

# from sklearn.cluster import DBSCAN

# # First, identify points that retained their original color (not turned black)
# original_color_indices = np.where(labels != 0)[0]  # Assuming label=0 was assigned to black

# # Extract the colors of these points
# original_color_colors = colors[original_color_indices]

# # Calculate average color and standard deviation for these points
# average_color = np.mean(original_color_colors, axis=0)
# std_dev_color = np.std(original_color_colors, axis=0)

# print(f"average color = {average_color}, standard deviation = {std_dev_color}")

# # Extract the points and colors of these points
# original_color_points = np.asarray(pcd.points)[original_color_indices]
# original_color_colors = colors[original_color_indices]

# # Standardize the points
# scaler = StandardScaler()
# original_color_points = scaler.fit_transform(original_color_points)

# # Apply DBSCAN to cluster these points
# dbscan = DBSCAN(eps=0.025, min_samples=10)  # Tweak these parameters as needed
# dbscan_labels = dbscan.fit_predict(original_color_points)

# # Initialize variables to store analysis results
# cluster_average_colors = []
# cluster_std_dev_colors = []

# # Analyze each cluster (excluding noise, labeled as -1)
# for cluster_label in set(dbscan_labels):
#     if cluster_label == -1:
#         continue  # Skip noise
#     cluster_indices = np.where(dbscan_labels == cluster_label)[0]
#     cluster_colors = original_color_colors[cluster_indices]
    
#     # Calculate average color and standard deviation for the cluster
#     average_color = np.mean(cluster_colors, axis=0)
#     std_dev_color = np.std(cluster_colors, axis=0)
    
#     cluster_average_colors.append(average_color)
#     cluster_std_dev_colors.append(std_dev_color)

# # Print out the analysis results
# num_clusters = len(cluster_average_colors)
# print(f"Number of clusters with original color: {num_clusters}")
# # for i, (avg_color, std_dev) in enumerate(zip(cluster_average_colors, cluster_std_dev_colors), start=1):
# #     print(f"Cluster {i}: Average Color = {avg_color}, Standard Deviation = {std_dev}")

# import matplotlib.pyplot as plt

# # Generate a color map with a unique color for each cluster
# color_map = plt.cm.get_cmap('jet', len(set(dbscan_labels)))

# # Initialize an array to store the colors of the points
# point_colors = np.zeros((len(original_color_points), 3))

# # Assign each cluster a unique color
# for cluster_label in set(dbscan_labels):
#     if cluster_label == -1:
#         continue  # Skip noise
#     cluster_indices = np.where(dbscan_labels == cluster_label)[0]
#     point_colors[cluster_indices] = color_map(cluster_label)[:3]  # Slice to keep only RGB, ignore alpha

# # Create a new Open3D point cloud for visualization
# clustered_pcd = o3d.geometry.PointCloud()
# clustered_pcd.points = o3d.utility.Vector3dVector(original_color_points)
# clustered_pcd.colors = o3d.utility.Vector3dVector(point_colors)

# # Visualize the clustered point cloud
# o3d.visualization.draw_geometries([clustered_pcd], window_name="Clustered Point Cloud")

# # Note: The eps parameter in DBSCAN affects the size and number of clusters
# # It may need adjustment based on the scale of your data and the desired granularity of clustering


Adjusted blossom points saved to adjusted_blossom_points.pcd
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


/opt/homebrew/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
KMeans segmented blossom points saved.
average color = [0.6105101  0.62712522 0.46248604], standard deviation = [0.0211496  0.03430516 0.04783854]
Number of clusters with original color: 893
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


In [7]:
import numpy as np
import open3d as o3d

def adjust_brightness(colors, target_brightness):
    # Calculate current brightness as the mean of RGB values
    current_brightness = np.mean(colors, axis=1)
    
    # Calculate the factor needed to adjust each point to the target brightness
    brightness_factor = target_brightness / (current_brightness + 1e-6)
    
    # Adjust colors
    adjusted_colors = colors * brightness_factor[:, np.newaxis]
    
    # Ensure the adjusted colors are within valid range
    adjusted_colors = np.clip(adjusted_colors, 0, 1)
    
    return adjusted_colors

# Load the point cloud of blossom points
pcd_path = 'nonground_cherry_1.pcd'
pcd = o3d.io.read_point_cloud(pcd_path)

# Extract colors from the loaded point cloud
all_colors = np.asarray(pcd.colors)

# Calculate the target brightness as the average brightness of all points
target_brightness = np.mean(np.mean(all_colors, axis=1))

# Adjust the brightness of all colors to the target brightness
normalized_colors = adjust_brightness(all_colors, target_brightness)

# Update the point cloud with normalized colors
pcd.colors = o3d.utility.Vector3dVector(normalized_colors)

# Save the adjusted point cloud back to a file or visualize it
adjusted_pcd_path = 'adjusted_blossom_points.pcd'
o3d.io.write_point_cloud(adjusted_pcd_path, pcd)
print(f"Adjusted blossom points saved to {adjusted_pcd_path}")

# Visualize the adjusted point cloud
o3d.visualization.draw_geometries([pcd], window_name="Adjusted Brightness Blossoms")

import numpy as np
import open3d as o3d
from sklearn.cluster import KMeans

# Load the brightness-adjusted point cloud
pcd_path = 'adjusted_blossom_points.pcd'
pcd = o3d.io.read_point_cloud(pcd_path)

# Extract colors and convert them to a suitable format for K-means
colors = np.asarray(pcd.colors)

# Apply K-means clustering to segment the points based on color
kmeans = KMeans(n_clusters=3, random_state=42).fit(colors)

# Use labels to determine color assignment
labels = kmeans.labels_

# Initialize an array for the new colors
new_colors = np.copy(colors)

# Calculate the greenness of each color
greenness = colors[:, 1] - (colors[:, 0] + colors[:, 2]) / 2

# Calculate the average greenness for each color class
average_greenness = [np.mean(greenness[labels == i]) for i in range(3)]

# Find the color class with the highest average greenness
greenest_class = np.argmax(average_greenness)

print(f"The greenest color class is {greenest_class}")

# Assign black color to one cluster and retain original color for the other
# Assuming we want to turn the first cluster (label=0) to black
for i, label in enumerate(labels):
    if label != greenest_class:
        new_colors[i] = [0, 0, 0]  # Set color to black for one class

# Update the point cloud colors
pcd.colors = o3d.utility.Vector3dVector(new_colors)

# Visualize the point cloud with one class as black and the other in original color
o3d.visualization.draw_geometries([pcd], window_name="KMeans Segmentation")

# Optionally, you can save this newly colored point cloud
o3d.io.write_point_cloud('kmeans_segmented_blossom_points.pcd', pcd)
print("KMeans segmented blossom points saved.")



from sklearn.cluster import DBSCAN

# First, identify points that retained their original color (not turned black)
original_color_indices = np.where(labels == greenest_class)[0]  # Assuming label=0 was assigned to black

# Extract the colors of these points
original_color_colors = colors[original_color_indices]

# Calculate average color and standard deviation for these points
average_color = np.mean(original_color_colors, axis=0)
std_dev_color = np.std(original_color_colors, axis=0)

print(f"average color = {average_color}, standard deviation = {std_dev_color}")

# Extract the points and colors of these points
original_color_points = np.asarray(pcd.points)[original_color_indices]
original_color_colors = colors[original_color_indices]

# Standardize the points
scaler = StandardScaler()
original_color_points = scaler.fit_transform(original_color_points)

# Apply DBSCAN to cluster these points
dbscan = DBSCAN(eps=0.03, min_samples=10)  # Tweak these parameters as needed
dbscan_labels = dbscan.fit_predict(original_color_points)

# Initialize variables to store analysis results
cluster_average_colors = []
cluster_std_dev_colors = []

# Analyze each cluster (excluding noise, labeled as -1)
for cluster_label in set(dbscan_labels):
    if cluster_label == -1:
        continue  # Skip noise
    cluster_indices = np.where(dbscan_labels == cluster_label)[0]
    cluster_colors = original_color_colors[cluster_indices]
    
    # Calculate average color and standard deviation for the cluster
    average_color = np.mean(cluster_colors, axis=0)
    std_dev_color = np.std(cluster_colors, axis=0)
    
    cluster_average_colors.append(average_color)
    cluster_std_dev_colors.append(std_dev_color)

# Print out the analysis results
num_clusters = len(cluster_average_colors)
print(f"Number of clusters with original color: {num_clusters}")
# for i, (avg_color, std_dev) in enumerate(zip(cluster_average_colors, cluster_std_dev_colors), start=1):
#     print(f"Cluster {i}: Average Color = {avg_color}, Standard Deviation = {std_dev}")

import matplotlib.pyplot as plt

# Generate a color map with a unique color for each cluster
color_map = plt.cm.get_cmap('jet', len(set(dbscan_labels)))

# Initialize an array to store the colors of the points
point_colors = np.zeros((len(original_color_points), 3))

# Assign each cluster a unique color
for cluster_label in set(dbscan_labels):
    if cluster_label == -1:
        continue  # Skip noise
    cluster_indices = np.where(dbscan_labels == cluster_label)[0]
    point_colors[cluster_indices] = color_map(cluster_label)[:3]  # Slice to keep only RGB, ignore alpha

# Create a new Open3D point cloud for visualization
clustered_pcd = o3d.geometry.PointCloud()
clustered_pcd.points = o3d.utility.Vector3dVector(original_color_points)
clustered_pcd.colors = o3d.utility.Vector3dVector(point_colors)

# Visualize the clustered point cloud
o3d.visualization.draw_geometries([clustered_pcd], window_name="Clustered Point Cloud")

# Initialize an array to store the colors of the points
original_point_colors = np.zeros((len(original_color_points), 3))

# Assign each point its original color
for i, label in enumerate(dbscan_labels):
    if label != -1:  # Skip noise
        original_point_colors[i] = original_color_colors[i]

# Create a new Open3D point cloud for visualization
original_color_pcd = o3d.geometry.PointCloud()
original_color_pcd.points = o3d.utility.Vector3dVector(original_color_points[dbscan_labels != -1])
original_color_pcd.colors = o3d.utility.Vector3dVector(original_point_colors[dbscan_labels != -1])

# Visualize the point cloud with original colors
o3d.visualization.draw_geometries([original_color_pcd], window_name="Original Color Point Cloud")

# Note: The eps parameter in DBSCAN affects the size and number of clusters
# It may need adjustment based on the scale of your data and the desired granularity of clustering


Adjusted blossom points saved to adjusted_blossom_points.pcd
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


/opt/homebrew/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


The greenest color class is 2
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
KMeans segmented blossom points saved.
average color = [0.56945113 0.59502062 0.39700013], standard deviation = [0.02066276 0.03098036 0.04174099]
Number of clusters with original color: 715
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


In [5]:
import numpy as np
import open3d as o3d

def adjust_brightness(colors, target_brightness):
    # Calculate current brightness as the mean of RGB values
    current_brightness = np.mean(colors, axis=1)
    
    # Calculate the factor needed to adjust each point to the target brightness
    brightness_factor = target_brightness / (current_brightness + 1e-6)
    
    # Adjust colors
    adjusted_colors = colors * brightness_factor[:, np.newaxis]
    
    # Ensure the adjusted colors are within valid range
    adjusted_colors = np.clip(adjusted_colors, 0, 1)
    
    return adjusted_colors

# Load the point cloud of blossom points
pcd_path = 'bloosom_points.pcd'
pcd = o3d.io.read_point_cloud(pcd_path)

# Extract colors from the loaded point cloud
all_colors = np.asarray(pcd.colors)

# Calculate the target brightness as the average brightness of all points
target_brightness = np.mean(np.mean(all_colors, axis=1))

# Adjust the brightness of all colors to the target brightness
normalized_colors = adjust_brightness(all_colors, target_brightness)

# Update the point cloud with normalized colors
pcd.colors = o3d.utility.Vector3dVector(normalized_colors)

# Save the adjusted point cloud back to a file or visualize it
adjusted_pcd_path = 'adjusted_blossom_points.pcd'
o3d.io.write_point_cloud(adjusted_pcd_path, pcd)
print(f"Adjusted blossom points saved to {adjusted_pcd_path}")

# Visualize the adjusted point cloud
o3d.visualization.draw_geometries([pcd], window_name="Adjusted Brightness Blossoms")

import numpy as np
import open3d as o3d
from sklearn.cluster import KMeans

# Load the brightness-adjusted point cloud
pcd_path = 'adjusted_blossom_points.pcd'
pcd = o3d.io.read_point_cloud(pcd_path)

# Extract colors and convert them to a suitable format for K-means
colors = np.asarray(pcd.colors)

# Apply K-means clustering to segment the points based on color
kmeans = KMeans(n_clusters=2, random_state=42).fit(colors)

# Use labels to determine color assignment
labels = kmeans.labels_

# Initialize an array for the new colors
new_colors = np.copy(colors)

# Assign black color to one cluster and retain original color for the other
# Assuming we want to turn the first cluster (label=0) to black
for i, label in enumerate(labels):
    if label == 0:
        new_colors[i] = [0, 0, 0]  # Set color to black for one class

# Update the point cloud colors
pcd.colors = o3d.utility.Vector3dVector(new_colors)

# Visualize the point cloud with one class as black and the other in original color
o3d.visualization.draw_geometries([pcd], window_name="KMeans Segmentation")

# Optionally, you can save this newly colored point cloud
o3d.io.write_point_cloud('kmeans_segmented_blossom_points.pcd', pcd)
print("KMeans segmented blossom points saved.")

from sklearn.cluster import DBSCAN

# First, identify points that retained their original color (not turned black)
original_color_indices = np.where(labels != 0)[0]  # Assuming label=0 was assigned to black

# Extract the colors of these points
original_color_colors = colors[original_color_indices]

# Calculate average color and standard deviation for these points
average_color = np.mean(original_color_colors, axis=0)
std_dev_color = np.std(original_color_colors, axis=0)

print(f"average color = {average_color}, standard deviation = {std_dev_color}")

# Extract the points and colors of these points
original_color_points = np.asarray(pcd.points)[original_color_indices]
original_color_colors = colors[original_color_indices]

# Standardize the points
scaler = StandardScaler()
original_color_points = scaler.fit_transform(original_color_points)

# Apply DBSCAN to cluster these points
dbscan = DBSCAN(eps=0.025, min_samples=10)  # Tweak these parameters as needed
dbscan_labels = dbscan.fit_predict(original_color_points)

# Initialize variables to store analysis results
cluster_average_colors = []
cluster_std_dev_colors = []

# Analyze each cluster (excluding noise, labeled as -1)
for cluster_label in set(dbscan_labels):
    if cluster_label == -1:
        continue  # Skip noise
    cluster_indices = np.where(dbscan_labels == cluster_label)[0]
    cluster_colors = original_color_colors[cluster_indices]
    
    # Calculate average color and standard deviation for the cluster
    average_color = np.mean(cluster_colors, axis=0)
    std_dev_color = np.std(cluster_colors, axis=0)
    
    cluster_average_colors.append(average_color)
    cluster_std_dev_colors.append(std_dev_color)

# Print out the analysis results
num_clusters = len(cluster_average_colors)
print(f"Number of clusters with original color: {num_clusters}")
# for i, (avg_color, std_dev) in enumerate(zip(cluster_average_colors, cluster_std_dev_colors), start=1):
#     print(f"Cluster {i}: Average Color = {avg_color}, Standard Deviation = {std_dev}")

import matplotlib.pyplot as plt

# Generate a color map with a unique color for each cluster
color_map = plt.cm.get_cmap('jet', len(set(dbscan_labels)))

# Initialize an array to store the colors of the points
point_colors = np.zeros((len(original_color_points), 3))

# Assign each cluster a unique color
for cluster_label in set(dbscan_labels):
    if cluster_label == -1:
        continue  # Skip noise
    cluster_indices = np.where(dbscan_labels == cluster_label)[0]
    point_colors[cluster_indices] = color_map(cluster_label)[:3]  # Slice to keep only RGB, ignore alpha

# Create a new Open3D point cloud for visualization
clustered_pcd = o3d.geometry.PointCloud()
clustered_pcd.points = o3d.utility.Vector3dVector(original_color_points)
clustered_pcd.colors = o3d.utility.Vector3dVector(point_colors)

# Visualize the clustered point cloud
o3d.visualization.draw_geometries([clustered_pcd], window_name="Clustered Point Cloud")

# Initialize an array to store the colors of the points
original_point_colors = np.zeros((len(original_color_points), 3))

# Assign each point its original color
for i, label in enumerate(dbscan_labels):
    if label != -1:  # Skip noise
        original_point_colors[i] = original_color_colors[i]

# Create a new Open3D point cloud for visualization
original_color_pcd = o3d.geometry.PointCloud()
original_color_pcd.points = o3d.utility.Vector3dVector(original_color_points[dbscan_labels != -1])
original_color_pcd.colors = o3d.utility.Vector3dVector(original_point_colors[dbscan_labels != -1])

# Visualize the point cloud with original colors
o3d.visualization.draw_geometries([original_color_pcd], window_name="Original Color Point Cloud")

# Note: The eps parameter in DBSCAN affects the size and number of clusters
# It may need adjustment based on the scale of your data and the desired granularity of clustering


Adjusted blossom points saved to adjusted_blossom_points.pcd
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


/opt/homebrew/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
KMeans segmented blossom points saved.
average color = [0.55708157 0.56546252 0.4389234 ], standard deviation = [0.02006236 0.03143084 0.04090002]
Number of clusters with original color: 1349
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


In [3]:
import numpy as np
import open3d as o3d
from sklearn.cluster import DBSCAN, KMeans
import matplotlib.pyplot as plt

# Assuming pcd is your point cloud and blossom_indices identifies the blossom points
points = np.asarray(pcd.points)
rgb_values = np.asarray(pcd.colors) * 255  # Assuming rgb_values are normalized between 0 and 1
blossom_points = points[blossom_indices]
blossom_colors = rgb_values[blossom_indices]

# Step 1: Spatial clustering with DBSCAN
dbscan = DBSCAN(eps=0.1, min_samples=10)  # Adjust eps and min_samples as needed
spatial_labels = dbscan.fit_predict(blossom_points)

segmented_colors = np.zeros((len(blossom_points), 3))

for spatial_label in np.unique(spatial_labels):
    if spatial_label == -1:  # Skip noise points
        continue
    cluster_indices = np.where(spatial_labels == spatial_label)[0]
    cluster_colors = blossom_colors[cluster_indices]

    # Step 2: Color clustering within each spatial cluster using K-means
    kmeans = KMeans(n_clusters=2, random_state=42).fit(cluster_colors)
    color_labels = kmeans.labels_
    cluster_centers = kmeans.cluster_centers_

    for i, label in enumerate(color_labels):
        if label == color_labels[0]:
            # Retain original color for points identified as green
            segmented_colors[cluster_indices[i]] = blossom_colors[cluster_indices[i]]
        else:
            # Use black for other points
            segmented_colors[cluster_indices[i]] = [0, 0, 0]

# Create a new Open3D point cloud for visualization
segmented_pcd = o3d.geometry.PointCloud()
segmented_pcd.points = o3d.utility.Vector3dVector(blossom_points)
segmented_pcd.colors = o3d.utility.Vector3dVector(segmented_colors / 255)  # Normalize colors to [0, 1] if necessary

# Visualize the segmented point cloud
o3d.visualization.draw_geometries([segmented_pcd], window_name="Segmented Blossom Clustering")


NameError: name 'blossom_indices' is not defined

In [19]:
import numpy as np
import open3d as o3d
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

# Assuming pcd is your point cloud and blossom_indices identifies the blossom points
points = np.asarray(pcd.points)
rgb_values = np.asarray(pcd.colors)

scalar = StandardScaler()
scaled_points = scalar.fit_transform(points)

# Extract blossom points and their colors
blossom_points = points[blossom_indices]
blossom_colors = rgb_values[blossom_indices]

# Apply K-means clustering on blossom colors
kmeans = KMeans(n_clusters=2, random_state=42).fit(blossom_colors)

# Labels for each blossom point
labels = kmeans.labels_

# Define new colors for visualization based on clusters
# Here, green for one cluster and black for another, adjust as needed
cluster_colors = np.array([
    [0, 1, 0],  # Green for the first cluster
    [0, 0, 0]   # Black for the second cluster
])

# Apply the cluster colors to the blossom points
segmented_blossom_colors = cluster_colors[labels]

# Create a new Open3D point cloud for the segmented blossom points
segmented_blossom_pcd = o3d.geometry.PointCloud()
segmented_blossom_pcd.points = o3d.utility.Vector3dVector(blossom_points)
segmented_blossom_pcd.colors = o3d.utility.Vector3dVector(segmented_blossom_colors)

# Visualize the segmented blossom point cloud
o3d.visualization.draw_geometries([segmented_blossom_pcd], window_name="Segmented Blossom Clustering")

# Optional: Evaluate the clusters to identify characteristics
for i in range(2):  # For each cluster
    cluster_mean_color = blossom_colors[labels == i].mean(axis=0)
    print(f"Cluster {i} mean color: {cluster_mean_color}")


/opt/homebrew/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
Cluster 0 mean color: [0.25847528 0.347514   0.4742711 ]
Cluster 1 mean color: [0. 0. 0.]


In [2]:

# Load the point cloud
pcd_path = 'nonground_cherry_1.pcd'
pcd = o3d.io.read_point_cloud(pcd_path)

# filter out the clear outliers
cl, ind = pcd.remove_statistical_outlier(nb_neighbors=20, std_ratio=2.0)
pcd = pcd.select_by_index(ind)

# Compute the axis-aligned bounding box for the point cloud
aabb = pcd.get_axis_aligned_bounding_box()
aabb.color = (1, 0, 0)  # Red color for visualization

# Visualize the point cloud and its bounding box
# o3d.visualization.draw_geometries([pcd, aabb])
category_to_color = {
    1: [0, 113/255, 188/255],  # Noise
    2: [216/255, 82/255, 24/255],  # Trunk
    3: [236/255, 176/255, 31/255],  # Branch
    4: [125/255, 46/255, 141/255],  # Leaf
    5: [118/255, 171/255, 47/255],  # Blossom
    6: [161/255, 19/255, 46/255]   # Ground
}

# load blossoms-v0.5.json
with open('blossoms-v0.5.json') as f:
    data = json.load(f)


point_annotations = data['dataset']['samples'][0]['labels']['ground-truth']['attributes']['point_annotations']
point_annotations_np = np.array(point_annotations)

In [4]:
def analyze_voxel_shape(points):
    """
    Analyze the shape of points within a voxel and return eigenvalues.
    :param points: NumPy array of points within the voxel.
    :return: Tuple containing a string indicating shape characteristic ('Spherical', 'Cylindrical', or 'Indeterminate')
             and the eigenvalues of the covariance matrix of points.
    """
    if len(points) < 4:  # Not enough points to form a 3D shape
        return 'Indeterminate', np.zeros(3)
    
    covariance_matrix = np.cov(points.T)
    eigenvalues, _ = np.linalg.eigh(covariance_matrix)
    sorted_eigenvalues = np.sort(eigenvalues)
    
    # Shape analysis based on eigenvalues
    if np.isclose(sorted_eigenvalues[0], sorted_eigenvalues[1]) and np.isclose(sorted_eigenvalues[1], sorted_eigenvalues[2]):
        shape = 'Spherical'
    elif np.isclose(sorted_eigenvalues[0], sorted_eigenvalues[1]) or np.isclose(sorted_eigenvalues[1], sorted_eigenvalues[2]):
        shape = 'Cylindrical'
    else:
        shape = 'Indeterminate'
    
    return shape, sorted_eigenvalues


def create_voxel_box(voxel_center, voxel_size, color=[1, 0, 0]):
    """
    Create a mesh box representing a voxel at the given center with the specified size and color.
    
    :param voxel_center: Center of the voxel (x, y, z)
    :param voxel_size: Size of the voxel
    :param color: Color of the voxel
    :return: Open3D geometry object representing the voxel box
    """
    mesh_box = o3d.geometry.TriangleMesh.create_box(width=voxel_size,
                                                    height=voxel_size,
                                                    depth=voxel_size)
    mesh_box.paint_uniform_color(color)
    mesh_box.translate(voxel_center - np.array([voxel_size / 2, voxel_size / 2, voxel_size / 2]))
    return mesh_box

def classify_voxel_based_on_eigenvalues(points):
    # Ensure points is a NumPy array for efficient computation
    points = np.array(points)
    
    # Calculate the covariance matrix of points within the voxel
    covariance_matrix = np.cov(points.T)
    
    # Compute eigenvalues from the covariance matrix
    eigenvalues, _ = np.linalg.eig(covariance_matrix)
    
    # Sort eigenvalues in ascending order
    eigenvalues = np.sort(eigenvalues)
    
    # Classification logic based on eigenvalues
    # This is a simplified example. You'll need to adjust the thresholds
    # and logic based on the geometric characteristics of your target objects.
    if eigenvalues[2] / eigenvalues[1] > 2 and eigenvalues[1] / eigenvalues[0] > 2:
        # Long and thin structure, more likely a branch or trunk
        if eigenvalues[0] < 0.01:  # Adjust threshold based on your data
            return 2
        else:
            return 3
    elif eigenvalues[2] / eigenvalues[0] < 2:
        # More spherical, could be a blossom
        return 4
    else:
        # Indeterminate or other structures
        return 2
# Function to predict voxel label based on eigenvalues
def predict_label(eigenvalues, avg_eigenvalues_dict, std_dev_eigenvalues_dict):
    # Measure the difference from each category's mean, normalized by the standard deviation
    category_scores = {}
    for category, avg_eigenvalues in avg_eigenvalues_dict.items():
        std_devs = std_dev_eigenvalues_dict[category]
        # Calculate a simple score by summing the absolute differences normalized by std dev
        score = sum(abs(eigenvalues - avg_eigenvalues) / std_devs)
        category_scores[category] = score
    # Predict label as the category with the minimum score
    predicted_label = min(category_scores, key=category_scores.get)
    return predicted_label

def annotation_to_label(annotation):
    if annotation == 4:  # Blossom
        return 'blossom'
    elif annotation == 2:  # Trunk
        return 'trunk'
    elif annotation == 3:  # Branch
        return 'branch'
    else:
        return None
    
    # Function to create a voxel box
def create_voxel_box(voxel_center, voxel_size, color=[1, 0, 0]):
    mesh_box = o3d.geometry.TriangleMesh.create_box(width=voxel_size, height=voxel_size, depth=voxel_size)
    mesh_box.paint_uniform_color(color)
    mesh_box.translate(voxel_center - np.array([voxel_size / 2, voxel_size / 2, voxel_size / 2]))
    return mesh_box

# Function to classify each voxel based on eigenvalues
def classify_voxel(eigenvalues, avg_eigenvalues_dict, std_dev_eigenvalues_dict):
    label_scores = {}
    for label, avg_eigenvalues in avg_eigenvalues_dict.items():
        std_dev_eigenvalues = std_dev_eigenvalues_dict[label]
        # Calculate Z-score for each eigenvalue dimension
        z_scores = np.abs((eigenvalues - avg_eigenvalues) / std_dev_eigenvalues)
        # Use sum of Z-scores as the label score
        label_scores[label] = np.sum(z_scores)
    # Assign the label with the lowest score
    predicted_label = min(label_scores, key=label_scores.get)
    return predicted_label

# Define color mapping for each label
label_color_mapping = {
    'blossom': [1, 0, 0],  # Red
    'branch': [0, 1, 0],  # Green
    'trunk': [0, 0, 1],  # Blue
    'indeterminate': [0.5, 0.5, 0.5]  # Gray
}

# Predict labels for voxels and visualize
def visualize_voxel_classification(voxel_points, voxel_size, min_bound, avg_eigenvalues_dict, std_dev_eigenvalues_dict):
    voxel_meshes = []
    for voxel_index, points in voxel_points.items():
        if len(points) > 3:  # Only consider voxels with enough points
            shape, eigenvalues = analyze_voxel_shape(np.array(points))
            label = classify_voxel(eigenvalues, avg_eigenvalues_dict, std_dev_eigenvalues_dict)
            color = label_color_mapping.get(label, [0.5, 0.5, 0.5])  # Default to gray if label not found
            voxel_center = np.array([
                min_bound[0] + (voxel_index[0] + 0.5) * voxel_size,
                min_bound[1] + (voxel_index[1] + 0.5) * voxel_size,
                min_bound[2] + (voxel_index[2] + 0.5) * voxel_size
            ])
            voxel_mesh = create_voxel_box(voxel_center, voxel_size, color)
            voxel_meshes.append(voxel_mesh)
    # Combine and visualize voxel meshes
    combined_mesh = o3d.geometry.TriangleMesh()
    for mesh in voxel_meshes:
        combined_mesh += mesh
    o3d.visualization.draw_geometries([combined_mesh], window_name="Voxel Classification")

    
    
# Define voxel size (sub-box size)
voxel_size = 0.01  # For example, 1cm^3

# # Get the dimensions of the bounding box
# min_bound = aabb.get_min_bound()  # Minimum coordinates (x_min, y_min, z_min)
# max_bound = aabb.get_max_bound()  # Maximum coordinates (x_max, y_max, z_max)

# # Calculate the number of divisions along each axis
# num_divisions_x = int(np.ceil((max_bound[0] - min_bound[0]) / voxel_size))
# num_divisions_y = int(np.ceil((max_bound[1] - min_bound[1]) / voxel_size))
# num_divisions_z = int(np.ceil((max_bound[2] - min_bound[2]) / voxel_size))

# # Convert point cloud to NumPy array for efficient computation
pcd_points = np.asarray(pcd.points)
pcd_colors = np.asarray(pcd.colors)  # If color analysis is needed

# # Initialize a 3D array to hold the count of points in each voxel
# voxel_grid = np.zeros((num_divisions_x, num_divisions_y, num_divisions_z))

# # Iterate through each voxel and count points
# for point in pcd_points:
#     # Determine the voxel in which the point falls
#     index_x = int((point[0] - min_bound[0]) / voxel_size)
#     index_y = int((point[1] - min_bound[1]) / voxel_size)
#     index_z = int((point[2] - min_bound[2]) / voxel_size)
    
#     # Increment the count of points in this voxel
#     voxel_grid[index_x, index_y, index_z] += 1
#     # Function to create a box mesh


# # Initialize a list to hold the voxel box meshes
# voxel_meshes = []

# # Iterate through the voxel grid to create boxes for non-empty voxels
# for ix in range(num_divisions_x):
#     for iy in range(num_divisions_y):
#         for iz in range(num_divisions_z):
#             if voxel_grid[ix, iy, iz] > 0:  # Check if the voxel contains any points
#                 # Calculate the center of the voxel
#                 voxel_center = np.array([min_bound[0] + (ix + 0.5) * voxel_size,
#                                          min_bound[1] + (iy + 0.5) * voxel_size,
#                                          min_bound[2] + (iz + 0.5) * voxel_size])
                
#                 # Create a voxel box mesh
#                 voxel_mesh = create_voxel_box(voxel_center, voxel_size, color=[0.8, 0.8, 0.8])  # Light gray color
#                 voxel_meshes.append(voxel_mesh)

# # Combine all voxel meshes into a single mesh (for efficiency)
# combined_mesh = voxel_meshes[0]
# for mesh in voxel_meshes[1:]:
#     combined_mesh += mesh



# Visualize the combined mesh
# o3d.visualization.draw_geometries([combined_mesh], window_name='Voxel Grid Visualization')
    
# Calculate the 75th percentile of points per non-empty voxel
# non_empty_voxels = voxel_grid[voxel_grid > 0]
# points_per_voxel_75th_percentile = np.percentile(non_empty_voxels, 75)

# Initialize labels for each voxel: 1 for above 75th percentile, 0 for below
# voxel_labels = np.zeros_like(voxel_grid)

# # Label voxels based on comparison with the 75th percentile
# for ix in range(num_divisions_x):
#     for iy in range(num_divisions_y):
#         for iz in range(num_divisions_z):
#             if voxel_grid[ix, iy, iz] > 0:  # Only consider non-empty voxels
#                 if voxel_grid[ix, iy, iz] > points_per_voxel_75th_percentile:
#                     voxel_labels[ix, iy, iz] = 1  # Above 75th percentile
#                 else:
#                     voxel_labels[ix, iy, iz] = 0  # Below 75th percentile

# # Colors for visualization
# color_above_mean = [0, 1, 0]  # Green for voxels above the mean
# color_below_mean = [1, 0, 0]  # Red for voxels below the mean

# # Initialize lists to hold voxel box meshes for different labels
# voxel_meshes_above_mean = []
# voxel_meshes_below_mean = []

# # Iterate through the labeled voxel grid and create box meshes with corresponding colors
# for ix in range(num_divisions_x):
#     for iy in range(num_divisions_y):
#         for iz in range(num_divisions_z):
#             if voxel_grid[ix, iy, iz] > 0:  # Only consider non-empty voxels
#                 voxel_center = np.array([min_bound[0] + (ix + 0.5) * voxel_size,
#                                          min_bound[1] + (iy + 0.5) * voxel_size,
#                                          min_bound[2] + (iz + 0.5) * voxel_size])
#                 if voxel_labels[ix, iy, iz] == 1:
#                     voxel_mesh = create_voxel_box(voxel_center, voxel_size, color=color_above_mean)
#                     voxel_meshes_above_mean.append(voxel_mesh)
#                 else:
#                     voxel_mesh = create_voxel_box(voxel_center, voxel_size, color=color_below_mean)
#                     voxel_meshes_below_mean.append(voxel_mesh)

# # Combine voxel meshes from both categories into single meshes for more efficient visualization
# combined_mesh_above_mean = voxel_meshes_above_mean[0]
# for mesh in voxel_meshes_above_mean[1:]:
#     combined_mesh_above_mean += mesh

# combined_mesh_below_mean = voxel_meshes_below_mean[0]
# for mesh in voxel_meshes_below_mean[1:]:
#     combined_mesh_below_mean += mesh

# print(len(voxel_meshes_above_mean))

# Visualize the combined meshes
# o3d.visualization.draw_geometries([combined_mesh_above_mean, combined_mesh_below_mean], window_name='Voxel Grid Visualization by Mean Point Count')

# Assuming `point_annotations_np` contains the label for each point
# and `pcd_points` is the array of point positions

# # Initialize arrays to hold point counts for blossom, branch, and trunk voxels
# blossom_voxel_counts = []
# branch_voxel_counts = []
# trunk_voxel_counts = []

# # Initialize a 3D grid to classify voxels
# voxel_classification = np.zeros((num_divisions_x, num_divisions_y, num_divisions_z))

# # Iterate over each point to classify voxels
# for i, point in enumerate(pcd_points):
#     label = point_annotations_np[i]
#     index_x = int((point[0] - min_bound[0]) / voxel_size)
#     index_y = int((point[1] - min_bound[1]) / voxel_size)
#     index_z = int((point[2] - min_bound[2]) / voxel_size)
    
#     # Classify the voxel based on the point label
#     if label == 4:  # Blossom
#         if voxel_classification[index_x, index_y, index_z] != 4:
#             voxel_classification[index_x, index_y, index_z] = 4
#             blossom_voxel_counts.append(1)
#         else:
#             blossom_voxel_counts[-1] += 1  # Increment the last count
#     elif label == 2:  # Trunk
#         if voxel_classification[index_x, index_y, index_z] != 2:
#             voxel_classification[index_x, index_y, index_z] = 2
#             trunk_voxel_counts.append(1)
#         else:
#             trunk_voxel_counts[-1] += 1
#     elif label == 3:  # Branch
#         if voxel_classification[index_x, index_y, index_z] != 3:
#             voxel_classification[index_x, index_y, index_z] = 3
#             branch_voxel_counts.append(1)
#         else:
#             branch_voxel_counts[-1] += 1

# mean_density_blossom = np.mean(blossom_voxel_counts) if blossom_voxel_counts else 0
# mean_density_branch = np.mean(branch_voxel_counts) if branch_voxel_counts else 0
# mean_density_trunk = np.mean(trunk_voxel_counts) if trunk_voxel_counts else 0

# print(f"Mean Density of Blossoms: {mean_density_blossom}")
# print(f"Mean Density of Branches: {mean_density_branch}")
# print(f"Mean Density of Trunks: {mean_density_trunk}")


# # Calculate the variance and standard deviation for each category
# variance_density_blossom = np.var(blossom_voxel_counts) if blossom_voxel_counts else 0
# std_dev_density_blossom = np.std(blossom_voxel_counts) if blossom_voxel_counts else 0

# variance_density_branch = np.var(branch_voxel_counts) if branch_voxel_counts else 0
# std_dev_density_branch = np.std(branch_voxel_counts) if branch_voxel_counts else 0

# variance_density_trunk = np.var(trunk_voxel_counts) if trunk_voxel_counts else 0
# std_dev_density_trunk = np.std(trunk_voxel_counts) if trunk_voxel_counts else 0

# print(f"Variance in Density of Blossoms: {variance_density_blossom}, Standard Deviation: {std_dev_density_blossom}")
# print(f"Variance in Density of Branches: {variance_density_branch}, Standard Deviation: {std_dev_density_branch}")
# print(f"Variance in Density of Trunks: {variance_density_trunk}, Standard Deviation: {std_dev_density_trunk}")



min_bound = pcd.get_min_bound()
max_bound = pcd.get_max_bound()
num_divisions_x = int(np.ceil((max_bound[0] - min_bound[0]) / voxel_size))
num_divisions_y = int(np.ceil((max_bound[1] - min_bound[1]) / voxel_size))
num_divisions_z = int(np.ceil((max_bound[2] - min_bound[2]) / voxel_size))

# Initialize dictionaries for voxel classification and points storage
voxel_classification = {}  # Maps voxel indices to labels based on contained points
voxel_points = {}  # Stores points for each voxel
# scalar = StandardScaler()
# scaled_points = scalar.fit_transform(pcd_points)
# Classify each point and aggregate in voxel_points
for i, point in enumerate(pcd_points):
    voxel_index = (int((point[0] - min_bound[0]) / voxel_size),
                   int((point[1] - min_bound[1]) / voxel_size),
                   int((point[2] - min_bound[2]) / voxel_size))
    label = annotation_to_label(point_annotations_np[i])  # Hypothetical function
    voxel_points.setdefault(voxel_index, []).append(point)
    # Update voxel classification based on annotations; refine this logic as needed
    voxel_classification[voxel_index] = label

# Analyze shape and calculate eigenvalues for each classified voxel
eigenvalues_dict = {'blossom': [], 'branch': [], 'trunk': []}
avg_eigenvalues_dict = {}
std_dev_eigenvalues_dict = {}

for voxel_index, points in voxel_points.items():
    if len(points) > 3:  # Ensure there are enough points for shape analysis
        _, eigenvalues = analyze_voxel_shape(np.array(points))
        label = voxel_classification.get(voxel_index)
        if label:
            eigenvalues_dict[label].append(eigenvalues)

# Calculate and print average eigenvalues for each label
for label, eigenvalues_list in eigenvalues_dict.items():
    if eigenvalues_list:
        avg_eigenvalues = np.mean(eigenvalues_list, axis=0)
        variance_eigenvalues = np.var(eigenvalues_list, axis=0)
        avg_eigenvalues_dict[label] = avg_eigenvalues
        std_dev_eigenvalues_dict[label] = np.sqrt(variance_eigenvalues)
        print(f"Average eigenvalues for {label}: {avg_eigenvalues}")
        print(f"Variance of eigenvalues for {label}: {variance_eigenvalues}")
        print('number of voxels:', len(eigenvalues_list))
    else:
        print(f"No data for {label}")


visualize_voxel_classification(voxel_points, voxel_size, min_bound, avg_eigenvalues_dict, std_dev_eigenvalues_dict)

Average eigenvalues for blossom: [1.18708757e-06 3.01201867e-06 8.08595091e-06]
Variance of eigenvalues for blossom: [1.22670131e-12 4.66562765e-12 2.53403754e-11]
number of voxels: 4909
Average eigenvalues for branch: [1.21096331e-06 3.04096311e-06 8.00344109e-06]
Variance of eigenvalues for branch: [1.26864744e-12 4.52948563e-12 2.25185635e-11]
number of voxels: 3900
Average eigenvalues for trunk: [1.19075248e-06 3.03206605e-06 8.03704044e-06]
Variance of eigenvalues for trunk: [1.19871695e-12 4.69290247e-12 2.49985518e-11]
number of voxels: 5989
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


In [ ]:

# # Create a dictionary to hold points for each voxel for shape analysis
# # Recalculate min and max bounds if necessary
# min_bound = np.min(pcd_points, axis=0)
# max_bound = np.max(pcd_points, axis=0)

# # Calculate the number of divisions along each axis again if necessary
# num_divisions_x = int(np.ceil((max_bound[0] - min_bound[0]) / voxel_size))
# num_divisions_y = int(np.ceil((max_bound[1] - min_bound[1]) / voxel_size))
# num_divisions_z = int(np.ceil((max_bound[2] - min_bound[2]) / voxel_size))

# # Reinitialize voxel_points dictionary to avoid KeyError
# voxel_points = {(x, y, z): [] for x in range(num_divisions_x) for y in range(num_divisions_y) for z in range(num_divisions_z)}

# # Populate voxel_points with actual points, ensuring indices are within bounds
# for i, point in enumerate(pcd_points):
#     index_x = min(max(int((point[0] - min_bound[0]) / voxel_size), 0), num_divisions_x - 1)
#     index_y = min(max(int((point[1] - min_bound[1]) / voxel_size), 0), num_divisions_y - 1)
#     index_z = min(max(int((point[2] - min_bound[2]) / voxel_size), 0), num_divisions_z - 1)
#     voxel_points[(index_x, index_y, index_z)].append(point)

# # Analyze shape for each voxel and classify
# voxel_shape_classification = {}
# for voxel_index, points in voxel_points.items():
#     if points:  # If there are points in the voxel
#         shape = analyze_voxel_shape(np.array(points))
#         voxel_shape_classification[voxel_index] = shape

# # Use different colors for different classifications
# shape_colors = {
#     'Spherical': [0, 1, 0],  # Green
#     'Cylindrical': [0, 0, 1],  # Blue
#     'Indeterminate': [1, 0, 0]  # Red
# }
# # Function to map annotations to labels (for demonstration purposes)
# def annotation_to_label(annotation):
#     if annotation == 4:  # Blossom
#         return 'blossom'
#     elif annotation == 2:  # Trunk
#         return 'trunk'
#     elif annotation == 3:  # Branch
#         return 'branch'
#     else:
#         return None

# # Calculate eigenvalues for each voxel's points and classify shape
# eigenvalues_dict = {'blossom': [], 'branch': [], 'trunk': []}

# # Iterate through each voxel to analyze shape and store eigenvalues
# for ix in range(num_divisions_x):
#     for iy in range(num_divisions_y):
#         for iz in range(num_divisions_z):
#             voxel_points = pcd_points[(voxel_grid[index_x, index_y, index_z] > 0)]
#             if len(voxel_points) > 3:
#                 shape, eigenvalues = analyze_voxel_shape(voxel_points)
#                 # Example: Directly using the first point's annotation as voxel's label, adjust as needed
#                 label = annotation_to_label(point_annotations_np[ix][iy][iz])
#                 if label:
#                     eigenvalues_dict[label].append(eigenvalues)

# # Calculate average eigenvalues for each classification
# for label, eigenvalues_list in eigenvalues_dict.items():
#     if eigenvalues_list:
#         avg_eigenvalues = np.mean(eigenvalues_list, axis=0)
#         print(f"Average eigenvalues for {label}: {avg_eigenvalues}")
#     else:
#         print(f"No data for {label}")

        


In [ ]:
# def classify_voxel_based_on_eigenvalues(points):
#     # Ensure points is a NumPy array for efficient computation
#     points = np.array(points)
    
#     # Calculate the covariance matrix of points within the voxel
#     covariance_matrix = np.cov(points.T)
    
#     # Compute eigenvalues from the covariance matrix
#     eigenvalues, _ = np.linalg.eig(covariance_matrix)
    
#     # Sort eigenvalues in ascending order
#     eigenvalues = np.sort(eigenvalues)
    
#     # Classification logic based on eigenvalues
#     # This is a simplified example. You'll need to adjust the thresholds
#     # and logic based on the geometric characteristics of your target objects.
#     if eigenvalues[2] / eigenvalues[1] > 2 and eigenvalues[1] / eigenvalues[0] > 2:
#         # Long and thin structure, more likely a branch or trunk
#         if eigenvalues[0] < 0.01:  # Adjust threshold based on your data
#             return 2
#         else:
#             return 3
#     elif eigenvalues[2] / eigenvalues[0] < 2:
#         # More spherical, could be a blossom
#         return 4
#     else:
#         # Indeterminate or other structures
#         return 2
# # Function to predict voxel label based on eigenvalues
# def predict_label(eigenvalues, avg_eigenvalues_dict, std_dev_eigenvalues_dict):
#     # Measure the difference from each category's mean, normalized by the standard deviation
#     category_scores = {}
#     for category, avg_eigenvalues in avg_eigenvalues_dict.items():
#         std_devs = std_dev_eigenvalues_dict[category]
#         # Calculate a simple score by summing the absolute differences normalized by std dev
#         score = sum(abs(eigenvalues - avg_eigenvalues) / std_devs)
#         category_scores[category] = score
#     # Predict label as the category with the minimum score
#     predicted_label = min(category_scores, key=category_scores.get)
#     return predicted_label

# # Visualization based on predicted labels
# for voxel_index, points in voxel_points.items():
#     if len(points) > 3:
#         _, eigenvalues = analyze_voxel_shape(np.array(points))
#         predicted_label = predict_label(eigenvalues, avg_eigenvalues_dict, std_dev_eigenvalues_dict)
#         color = category_to_color(predicted_label)
#         voxel_center = np.array([
#             min_bound[0] + (voxel_index[0] + 0.5) * voxel_size,
#             min_bound[1] + (voxel_index[1] + 0.5) * voxel_size,
#             min_bound[2] + (voxel_index[2] + 0.5) * voxel_size
#         ])
#         voxel_mesh = create_voxel_box(voxel_center, voxel_size, color)
#         voxel_meshes.append(voxel_mesh)

# # Combine and visualize all voxel meshes
# combined_mesh = voxel_meshes[0]
# for mesh in voxel_meshes[1:]:
#     combined_mesh += mesh
# o3d.visualization.draw_geometries([combined_mesh], window_name="Voxel Classification Visualization")


In [ ]:

import numpy as np
import open3d as o3d

# Assuming the necessary preliminary steps are done and eigenvalues_dict is populated

# Function to classify voxels based on eigenvalues within 2 standard deviations
def classify_voxel(eigenvalues, avg_eigenvalues_dict, std_dev_eigenvalues_dict):
    for label, avg_eigenvalues in avg_eigenvalues_dict.items():
        std_dev_eigenvalues = std_dev_eigenvalues_dict[label]
        if np.all(np.abs(eigenvalues - avg_eigenvalues) <= 2 * std_dev_eigenvalues):
            return label
    return 'indeterminate'

# Function to visualize voxels
def visualize_voxels(voxel_labels, voxel_size, min_bound):
    voxel_meshes = []
    for voxel_index, label in voxel_labels.items():
        center = np.array([min_bound[0] + (voxel_index[0] + 0.5) * voxel_size,
                           min_bound[1] + (voxel_index[1] + 0.5) * voxel_size,
                           min_bound[2] + (voxel_index[2] + 0.5) * voxel_size])
        color = [0.5, 0.5, 0.5]  # Default color for indeterminate
        if label == 'blossom':
            color = [1, 0, 0]
        elif label == 'branch':
            color = [0, 1, 0]
        elif label == 'trunk':
            color = [0, 0, 1]
        mesh = create_voxel_box(center, voxel_size, color)
        voxel_meshes.append(mesh)
    o3d.visualization.draw_geometries(voxel_meshes, window_name="Voxel Classification")

# Calculate average and standard deviation for each category
avg_eigenvalues_dict = {label: np.mean(values, axis=0) for label, values in eigenvalues_dict.items()}
std_dev_eigenvalues_dict = {label: np.std(values, axis=0) for label, values in eigenvalues_dict.items()}

# Predict labels for each voxel and visualize
voxel_labels = {}
for voxel_index, points in voxel_points.items():
    if len(points) > 3:  # Only consider voxels with enough points
        _, eigenvalues = analyze_voxel_shape(np.array(points))
        label = classify_voxel(eigenvalues, avg_eigenvalues_dict, std_dev_eigenvalues_dict)
        voxel_labels[voxel_index] = label

visualize_voxels(voxel_labels, voxel_size, min_bound)


In [ ]:

# Function to calculate mean and standard deviation for eigenvalues
def calc_mean_std(eigenvalues_list):
    mean_eigenvalues = np.mean(eigenvalues_list, axis=0)
    std_dev_eigenvalues = np.std(eigenvalues_list, axis=0)
    return mean_eigenvalues, std_dev_eigenvalues

# Calculate mean and standard deviation for each category
mean_std_dict = {label: calc_mean_std(eigenvalues_list) for label, eigenvalues_list in eigenvalues_dict.items()}

# Function to classify voxels based on eigenvalues
def classify_voxel(eigenvalues, mean_std_dict):
    for label, (mean_eigenvalues, std_dev_eigenvalues) in mean_std_dict.items():
        if np.all(np.abs(eigenvalues - mean_eigenvalues) <= 2 * std_dev_eigenvalues):
            return label
    return None

# Classify each voxel and store the result
voxel_labels = {}
for voxel_index, points in voxel_points.items():
    if len(points) > 3:  # Ensure there are enough points for shape analysis
        _, eigenvalues = analyze_voxel_shape(np.array(points))
        voxel_labels[voxel_index] = classify_voxel(eigenvalues, mean_std_dict)

# Visualization
voxel_meshes = []
for voxel_index, label in voxel_labels.items():
    voxel_center = np.array([min_bound[0] + (voxel_index[0] + 0.5) * voxel_size,
                             min_bound[1] + (voxel_index[1] + 0.5) * voxel_size,
                             min_bound[2] + (voxel_index[2] + 0.5) * voxel_size])
    color = [0.5, 0.5, 0.5]  # Default color for indeterminate
    if label == 'blossom':
        color = [1, 0, 0]  # Red for blossom
    elif label == 'branch':
        color = [0, 1, 0]  # Green for branch
    elif label == 'trunk':
        color = [0, 0, 1]  # Blue for trunk
    voxel_mesh = create_voxel_box(voxel_center, voxel_size, color)
    voxel_meshes.append(voxel_mesh)

# Combine and visualize
combined_mesh = voxel_meshes[0]
for mesh in voxel_meshes[1:]:
    combined_mesh += mesh
o3d.visualization.draw_geometries([combined_mesh], window_name="Voxel Classification")
# Initialize lists to accumulate RGB values for each category
blossom_rgb_values = []
branch_rgb_values = []
trunk_rgb_values = []

# Iterate over each point to classify voxels and accumulate RGB values
for i, point in enumerate(pcd_points):
    label = point_annotations_np[i]
    index_x = int((point[0] - min_bound[0]) / voxel_size)
    index_y = int((point[1] - min_bound[1]) / voxel_size)
    index_z = int((point[2] - min_bound[2]) / voxel_size)
    
    # Accumulate RGB values based on the point label
    if label == 4:  # Blossom
        blossom_rgb_values.append(pcd_colors[i])
    elif label == 2:  # Trunk
        trunk_rgb_values.append(pcd_colors[i])
    elif label == 3:  # Branch
        branch_rgb_values.append(pcd_colors[i])

# Convert lists to numpy arrays for efficient computation
blossom_rgb_values = np.array(blossom_rgb_values)
branch_rgb_values = np.array(branch_rgb_values)
trunk_rgb_values = np.array(trunk_rgb_values)

# Calculate the mean RGB values for each category
mean_rgb_blossom = np.mean(blossom_rgb_values, axis=0) if blossom_rgb_values.size else [0, 0, 0]
mean_rgb_branch = np.mean(branch_rgb_values, axis=0) if branch_rgb_values.size else [0, 0, 0]
mean_rgb_trunk = np.mean(trunk_rgb_values, axis=0) if trunk_rgb_values.size else [0, 0, 0]

print(f"Mean RGB of Blossoms: {mean_rgb_blossom}")
print(f"Mean RGB of Branches: {mean_rgb_branch}")
print(f"Mean RGB of Trunks: {mean_rgb_trunk}")


In [3]:
import sympy as sp

# Define symbolic variables
t = sp.symbols('t')
x = sp.Function('x')(t)
y = sp.Function('y')(t)
z = sp.Function('z')(t)
dx = x.diff(t)
dy = y.diff(t)
dz = z.diff(t)
lambda_1, lambda_2, r, R = sp.symbols('lambda_1 lambda_2 r R')

# Define the Lagrangian with the inequality constraints incorporated
L = sp.sqrt(dx**2 + dy**2 + dz**2)
constraint1 = r**2 - x**2 - y**2 - z**2  # r^2 <= x^2 + y^2 + z^2
constraint2 = x**2 + y**2 + z**2 - R**2  # x^2 + y^2 + z^2 <= R^2

# Incorporate constraints using Lagrange multipliers
Lagrangian = L + lambda_1*constraint1 + lambda_2*constraint2

# Compute the Euler-Lagrange equations
euler_lagrange_eqns = []
for var in [x, y, z]:
    dL_dvar = Lagrangian.diff(var)
    dL_ddotvar = Lagrangian.diff(var.diff(t))
    euler_lagrange_eqn = dL_dvar - sp.diff(dL_ddotvar, t)
    euler_lagrange_eqns.append(euler_lagrange_eqn)

# Display the Euler-Lagrange equations and their derivatives set to zero
euler_lagrange_eqns_simplified = [eqn.simplify() for eqn in euler_lagrange_eqns]
euler_lagrange_eqns_simplified


[(2*(-lambda_1 + lambda_2)*(Derivative(x(t), t)**2 + Derivative(y(t), t)**2 + Derivative(z(t), t)**2)**2*x(t) + (Derivative(x(t), t)*Derivative(x(t), (t, 2)) + Derivative(y(t), t)*Derivative(y(t), (t, 2)) + Derivative(z(t), t)*Derivative(z(t), (t, 2)))*sqrt(Derivative(x(t), t)**2 + Derivative(y(t), t)**2 + Derivative(z(t), t)**2)*Derivative(x(t), t) - (Derivative(x(t), t)**2 + Derivative(y(t), t)**2 + Derivative(z(t), t)**2)**(3/2)*Derivative(x(t), (t, 2)))/(Derivative(x(t), t)**2 + Derivative(y(t), t)**2 + Derivative(z(t), t)**2)**2,
 (2*(-lambda_1 + lambda_2)*(Derivative(x(t), t)**2 + Derivative(y(t), t)**2 + Derivative(z(t), t)**2)**2*y(t) + (Derivative(x(t), t)*Derivative(x(t), (t, 2)) + Derivative(y(t), t)*Derivative(y(t), (t, 2)) + Derivative(z(t), t)*Derivative(z(t), (t, 2)))*sqrt(Derivative(x(t), t)**2 + Derivative(y(t), t)**2 + Derivative(z(t), t)**2)*Derivative(y(t), t) - (Derivative(x(t), t)**2 + Derivative(y(t), t)**2 + Derivative(z(t), t)**2)**(3/2)*Derivative(y(t), (t, 2)